# ILP Description

## Abstract
Liquiidty provisioning is a stochastic optimal control problem
It has a well defined a untility function which we need to miaximaize as obejctive function of probelm
Some applications of RL in finance include trading bots, risk optimization and portfolio managemnet

## Algorithims
1. **Proximal Policy Optimization (PPO)**:
   - **Why?** PPO is known for its stability and robustness. It's a policy gradient method, which means it directly optimizes the policy without needing to estimate value functions, making it more stable
   - **Challenges:** Might require more samples compared to off-policy methods

2. **Soft Actor-Critic (SAC)**:
   - **Why?** SAC can handle continuous action spaces, and its entropy regularization encourages exploration. It's also off-policy, meaning it can learn from past experiences more efficiently.
   - **Challenges:** Adapting SAC for the specific action space of Uniswap v3 (i.e., defining liquidity ranges) might be non-trivial.

3. **Deep Deterministic Policy Gradient (DDPG) or Twin Delayed DDPG (TD3)**:
   - **Why?** Both algorithms are designed for continuous action spaces. TD3 also introduces mechanisms to reduce value overestimation, which can improve stability.
   - **Challenges:** Like SAC, adapting these to the liquidity provisioning problem might require careful design.

4. **Q-Learning or Deep Q-Networks (DQN)**:
   - **Why?** While traditionally designed for discrete action spaces, they can be adapted for liquidity provisioning by discretizing the action space (i.e., having fixed liquidity ranges).
   - **Challenges:** Discretization might not capture all nuances of liquidity provisioning in Uniswap v3.

## Customized Reward Function based on LP's prefernces
In fact, personalizing the reward function according to the LP's preferences is one of the primary advantages of using an RL approach for liquidity provisioning. Each LP might have different priorities and risk tolerances, and the reward function should reflect that. Here's how you can adapt the reward function based on LP preferences:

1. **Risk Tolerance**: 
    - **Risk-Averse**: If an LP is risk-averse, you can increase the weight \( w_2 \) to heavily penalize any increase in impermanent loss. You might also want to have a higher penalty in \( R_{\text{safety}} \) to ensure that liquidity doesn't drop to risky levels.
    - **Risk-Taking**: For a more risk-taking LP, you might reduce the weight on \( R_{\text{IL}} \) and \( R_{\text{safety}} \) to allow the agent to explore potentially higher reward strategies that come with increased risk.

2. **Profit Motivation**:
    - **High**: If the primary goal is to maximize profits, increase the weight \( w_1 \) associated with fees. You might also want to emphasize liquidity utilization by increasing \( w_4 \), as high utilization generally leads to higher fee earnings.
    - **Moderate**: For a balanced approach, the weights can be more evenly distributed.

3. **Cost Sensitivity**: 
    - If an LP is particularly sensitive to costs (like gas fees), increase the weight \( w_3 \) to make sure the agent is more conservative in making liquidity adjustments that involve on-chain transactions.

4. **Liquidity Utilization**:
    - **Maximize**: If an LP wants their funds to be utilized to the maximum extent, increase \( w_4 \) to encourage the agent to provide liquidity in high volume areas.
    - **Balanced**: A balanced weight ensures that the agent doesn't overly concentrate liquidity in high volume areas, spreading risk.

5. **Custom Components**: LPs might have other specific preferences or goals not captured in the standard components. For instance:
    - **Diversification**: An LP might want to spread their liquidity across multiple pools or tokens. You can introduce a reward component that rewards diversification.
    - **Long-term positions**: If an LP has a long-term bullish view on a particular token, the reward function can be designed to favor providing liquidity to that token's pools.

Once the LP's preferences are defined, you can adjust the weights \( w_1, w_2, \ldots, w_5 \) (and introduce new components if needed) to reflect those preferences in the reward function.

Remember, the key is iteration. Once you've set a reward function based on the LP's preferences, you'll want to monitor the agent's performance and potentially make further adjustments to better align with the LP's goals over time.

#### State Space:
1. **Pool Data**: This would include current and historical data for all Uniswap pools. Features could include the current price, liquidity concentration, volume, fees collected, recent price movements, etc.
2. **Market Data**: Broader market data can provide context. This can include overall market sentiment, price movements of major crypto assets, news or events that might affect the crypto market, etc.
3. **Agent's Inventory**: Current holdings of the LP, including assets not yet committed to any pool.

#### Action Space:
1. **Select Pool**: Choose which pool to provide liquidity to.
2. **Select Amount**: Decide the amount of assets to provide as liquidity.
3. **Select Tick Range**: Define the price range within which the LP wants to earn fees.
4. **Re-adjust Position**: Move the liquidity to a different tick range based on changing market conditions.
5. **Do Nothing**: Sometimes, the best action might be to maintain the current position without making changes.
6. **Sit Idle**: Decide to not participate in liquidity provisioning, either waiting for a better opportunity or based on some other strategy.

## Observation State representation
## Features Engineering

### 1. **Price Statistics**:
   - **Current Price**: The current price of the trading pair.
   - **Price Volatility**: Standard deviation of price changes over a recent time window.
   - **Price Trend**: Slope of a linear regression on recent price data, indicating the direction and strength of the trend.

### 2. **Liquidity Distribution**:
   - **Total Liquidity**: Total liquidity across all tick ranges.
   - **Liquidity Concentration**: Measures of how liquidity is concentrated across different tick ranges (e.g., Gini coefficient, entropy).
   - **Liquidity in Current Range**: Liquidity within the current price range.

### 3. **Trading Volume Statistics**:
   - **Total Volume**: Total trading volume over a recent time window.
   - **Volume Distribution**: Distribution of trading volume across different tick ranges.
   - **Volume Momentum**: Rate of change in trading volume.

### 4. **Swap Data**:
   - **Swap Frequency**: Number of swaps occurring in a given time window.
   - **Average Swap Size**: Average size of swaps over a recent time window.
   - **Swap Direction**: Ratio of buy to sell swaps, indicating market sentiment.

### 5. **Fee Metrics**:
   - **Total Fees Earned**: Total fees earned by LPs over a recent time window.
   - **Fee Rate**: The fee rate associated with the current liquidity range.

### 6. **External Market Indicators**:
   - **Correlated Asset Prices**: Prices of other assets that are correlated with the trading pair.
   - **Market Sentiment Indicators**: Metrics derived from social media, news, or other sources that may influence trading behavior.

### 7. **Historical Data and Moving Averages**:
   - **Moving Averages**: Various moving averages of price, volume, etc., over different time windows.
   - **Historical Price Levels**: Past support and resistance levels that might influence trading behavior.

### 8. **Network Metrics**:
   - **Gas Prices**: Current Ethereum gas prices, which may influence trading and liquidity provisioning behavior.
   - **Transaction Queue**: Information about the pending transaction queue on the Ethereum network.

# Tokenspice Setup


## Steps
brownie compile (Inside directory which contains contracts folder to compile contracts in contracts folder)
tsp comiple (tsp interface which calls compile.sh file which calls brownie compile command in each folder to compile)
tsp run netlist_path (Creates and compiles agents in netlist)
rm -rf outdir2_csv; tsp run netlists/uniswapV3/netlist.py outdir_csv
tsp plot netlists/UniswapV3Netlist.py outdir_csv outdir_png

### Step 1: Install Prerequisites

1. **Install Python 3.8.5 or higher**: Download it from [here](https://www.python.org/downloads/).

2. **Install Node.js and npm**: Download it from [here](https://nodejs.org/en/download/).



### Step 2: Clone TokenSPICE Repository


### Step 3: Set Up Virtual Environment and Install Dependencies

1. Navigate to the cloned `tokenspice` directory in the terminal.
    ```bash
    cd tokenspice
    ```

2. Create a Python virtual environment:
    ```bash
    python3 -m venv venv
    ```

3. Activate the virtual environment:
    ```bash
    .\venv\Scripts\activate
    ```

4. Install the required Python packages:
    ```bash
    pip install -r requirements.txt
    ```

5. Install Brownie packages:
    ```bash
    .\brownie-install.sh
    ```

6. Install Ganache globally via npm:
    ```bash
    npm install ganache --global
    ```

### Step 4: Run Ganache

3. Run this command to start Ganache:
    ```bash
    .\ganache.py
    ```

### Step 5: Prepare Uniswap V3 Netlist and Agents

### Step 1: Add Uniswap V3 Contracts

1. **Download or clone Uniswap V3 contracts**: You can get them from their [GitHub repository](https://github.com/Uniswap/v3-core).

2. **Place the contracts in TokenSPICE**: Create a new folder under `tokenspice/contracts` and place your Uniswap V3 contracts there.

### Step 2: Compile Uniswap V3 Contracts

tsp compile
    ```

### Step 3: Create Agents for Uniswap V3

1. **Navigate to the `agents/` directory in `tokenspice`**.

2. **Create new Python files for Uniswap V3 agents**, like `UniswapV3LiquidityProviderAgent.py` or `UniswapV3SwapperAgent.py`.

### Step 4: Update Your Netlist

1. **Navigate to your Uniswap V3 netlist folder**, e.g., `netlists/uniswapv3/`.

2. **Open `netlist.py`**.

3. **Add your new agents to the netlist**. Import the agents and add instances of them to `SimState`.

### Step 5: Implement Simulation Logic

1. In the `takeStep()` method of your agents, **implement the logic for interacting with Uniswap V3 contracts**. This might include providing liquidity, making swaps, etc.
### Step 6: Run Your Simulation

    ```bash
    tsp run netlists/uniswapv3/netlist.py outdir_csv
    ```

### Step 7: View Results

1. Check the `outdir_csv` folder for the simulation results.

    ```bash
    tsp plot netlists/uniswapv3/netlist.py outdir_csv outdir_png
    ```

3. Open the `outdir_png` folder to view the plots.


# get Uniswap pool data to sync pools

In [ ]:
import requests
import pandas as pd

def fetch_uniswap_data_to_df():
    url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
    query = '''
    {
      mints(first: 10, orderBy: amountUSD, orderDirection: desc) {
        id
        tickLower
        tickUpper
        timestamp
        amountUSD
        owner
        logIndex
        origin
        sender
        amount
        amount0
        amount1
        pool {
          id
          feeTier
          token0 {
            name
            symbol
          }
          token1 {
            name
            symbol
          }
          feesUSD
          feeGrowthGlobal0X128
          feeGrowthGlobal1X128
          collectedFeesUSD
        }
      }
    }
    '''
    response = requests.post(url, json={'query': query})
    if response.status_code == 200:
        data = response.json()['data']['mints']
        df = pd.json_normalize(data, sep='_')
        return df
    else:
        print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")
        return None

# Usage
df = fetch_uniswap_data_to_df()
df.head(10)

In [ ]:

def fetch_pool_data(pool_id= "0x4e68ccd3e89f51c3074ca5072bbac773960dfa36", UNISWAP_V3_SUBGRAPH_URL = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'):
    # Fetching liquidity positions
    liquidity_query = """
    {
      positions(where: { pool: "%s" ,liquidity_gt: "0"}) {
        liquidity
        tickLower {
          price0
          tickIdx
        }
        tickUpper {
          price0
          tickIdx
        }
      }
    }
    """ % pool_id
    liquidity_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': liquidity_query}).json()

    tick_query="""
     {
     ticks(where: {liquidityGross_gt: "0", liquidityNet_gt: "0", pool: "%s"}) {
     
      tickIdx
      liquidityGross
      liquidityNet
      
    }
  }
  """% pool_id
    tick_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': tick_query}).json()
    # Fetching pool price, volume, fees, and reserves
    global_query = """
    {
      pools(where: { id: "%s" }) {
        token0Price
        token1Price
        volumeUSD
        feesUSD
        totalValueLockedToken0
        totalValueLockedToken1
      }
    }
    """ % pool_id
    global_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': global_query}).json()

    # Fetching swaps and trades
    swaps_query = """
    {
      swaps(where: { pool: "%s" }, first: 10, orderBy: timestamp, orderDirection: desc) {
        amount0
        amount1
        amountUSD
        timestamp
      }
    }
    """ % pool_id
    swaps_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': swaps_query}).json()

        # Constructing the state representation with float data type
    state = {
        'positions': [(float(position['tickLower']['tickIdx']), float(position['tickUpper']['tickIdx']), float(position['liquidity'])) for position in liquidity_data['data']['positions']],
        'pool_price': float(global_data['data']['pools'][0]['token1Price']),
        'ticks': [(float(tick['tickIdx']), float(tick['liquidityGross']), float(tick['liquidityNet'])) for tick in tick_data['data']['ticks']]
    }

    return state
state=fetch_pool_data()
# get events data

In [ ]:

df_positions=pd.DataFrame(state['positions'])
df_positions.head()
len(df_positions)


100

In [ ]:
df_ticks=pd.DataFrame(state['ticks'])
df_ticks.head()
#num_rows = len(df_ticks)
#num_rows

,0,1,2
0,-184260.0,5.554716e+12,5.554716e+12
1,-184320.0,4.944799e+12,4.944799e+12
2,-186060.0,1.368139e+12,1.368139e+12
3,-186240.0,1.280684e+12,1.280684e+12
4,-186480.0,7.737295e+11,7.737295e+11


# Load local pool

In [ ]:
import sys
sys.path.append('/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice')

import os
os.environ["PATH"] += ":."

from util.constants import BROWNIE_PROJECTUniV3, GOD_ACCOUNT
from util.constants import BROWNIE_PROJECTUniV3, GOD_ACCOUNT
from util.base18 import toBase18, fromBase18,fromBase128,price_to_valid_tick,price_to_raw_tick,price_to_sqrtp,sqrtp_to_price,tick_to_sqrtp,liquidity0,liquidity1,eth
import brownie
from web3 import Web3
import json
import math
import random
from brownie.exceptions import VirtualMachineError

class UniV3Model():
    def __init__(self, token0='token0', token1='token1', token0_decimals=18, token1_decimals=18, supply_token0=1e18, supply_token1=1e18, fee_tier=3000, initial_pool_price=1,deployer=GOD_ACCOUNT,sync_pool_with_liq=True,sync_pool_with_ticks=False,sync_pool_with_positions=False,sync_pool_with_events=False, state=None, initial_liquidity_amount=1000000):
        self.deployer = deployer
        self.token0_name = token0
        self.token1_name = token1
        self.token0_symbol = token0
        self.token1_symbol = token1
        self.token0_decimals = token0_decimals
        self.token1_decimals = token1_decimals
        self.supply_token0 = supply_token0
        self.supply_token1 = supply_token1
        self.fee_tier = fee_tier
        self.initial_pool_price = initial_pool_price
        self.sync_pool_with_liq=sync_pool_with_liq
        self.sync_pool_with_ticks=sync_pool_with_ticks
        self.sync_pool_with_positions=sync_pool_with_positions
        self.sync_pool_with_events=sync_pool_with_events
        self.initial_liquidity_amount=initial_liquidity_amount
        self.pool_id = f"{token0}_{token1}_{fee_tier}"
        

        w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
        self.base_fee = w3.eth.getBlock('latest')['baseFeePerGas']
        
        self.deploy_load_tokens()
        self.deploy_load_pool()

    def load_addresses(self):
        try:
            with open("model_storage/token_pool_addresses.json", "r") as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            return {}

    def save_addresses(self, addresses):
        with open("model_storage/token_pool_addresses.json", "w") as f:
            json.dump(addresses, f)

    def deploy_load_tokens(self):
        SimpleToken = BROWNIE_PROJECTUniV3.Simpletoken
        addresses = self.load_addresses()
        pool_addresses = addresses.get(self.pool_id, {})

        # This function deploys a token and saves its address in the JSON file
        def deploy_and_save_token(name, symbol, decimals, supply, key):
            token = SimpleToken.deploy(name, symbol, decimals, toBase18(supply),  {'from': self.deployer, 'gas_price': self.base_fee + 1})
            print(f"New {symbol} token deployed at {token.address}")
            pool_addresses[key] = token.address
            addresses[self.pool_id] = pool_addresses
            self.save_addresses(addresses)
            return token

        # Load or deploy token1
        if "token1_address" in pool_addresses:
            self.token1 = SimpleToken.at(pool_addresses["token1_address"])
        else:
            self.token1 = deploy_and_save_token(self.token1_name, self.token1_symbol, self.token1_decimals, self.supply_token1, "token1_address")

        # Load or deploy token0
        if "token0_address" in pool_addresses:
            self.token0 = SimpleToken.at(pool_addresses["token0_address"])
        else:
            self.token0 = deploy_and_save_token(self.token0_name, self.token0_symbol, self.token0_decimals, self.supply_token0, "token0_address")
            # Ensure token0 address is less than token1 address
            while int(self.token0.address, 16) >= int(self.token1.address, 16):
                self.token0 = deploy_and_save_token(self.token0_name, self.token0_symbol, self.token0_decimals, self.supply_token0, "token0_address")


    def deploy_load_pool(self):
        UniswapV3Factory = BROWNIE_PROJECTUniV3.UniswapV3Factory
        UniswapV3Pool = BROWNIE_PROJECTUniV3.UniswapV3Pool
        addresses = self.load_addresses()
        pool_addresses = addresses.get(self.pool_id, {})

        if "pool_address" in pool_addresses:
            self.pool = UniswapV3Pool.at(pool_addresses["pool_address"])
            print(f"Existing pool:{self.pool_id} having pool address: {self.pool} loaded")
        else:
            self.factory = UniswapV3Factory.deploy( {'from': self.deployer, 'gas_price': self.base_fee + 1})
            pool_creation_txn = self.factory.createPool(self.token0.address, self.token1.address, self.fee_tier,  {'from': self.deployer, 'gas_price': self.base_fee + 1})
            self.pool_address = pool_creation_txn.events['PoolCreated']['pool']
            print(pool_creation_txn.events)
            self.pool = UniswapV3Pool.at(self.pool_address)

            sqrtPriceX96 = price_to_sqrtp(self.initial_pool_price)
            tx_receipt=self.pool.initialize(sqrtPriceX96,  {'from': self.deployer, 'gas_price': self.base_fee + 100000})
            print(tx_receipt.events)

            pool_addresses["pool_address"] = self.pool_address
            addresses[self.pool_id] = pool_addresses
            self.save_addresses(addresses)
            
            self.sync_pool_state()


    def ensure_token_order(self):
        # Check if token0's address is greater than token1's address
        if int(self.token0.address, 16) > int(self.token1.address, 16):
            SimpleToken = BROWNIE_PROJECTUniV3.Simpletoken

            # Continue deploying token0 until its address is less than token1's address
            while True:
                new_token0 = SimpleToken.deploy(self.token0_name, self.token0_symbol, self.token0_decimals, self.supply_token0, {'from': self.deployer, 'gas_price': self.base_fee + 1})
                if int(new_token0.address, 16) < int(self.token1.address, 16):
                    break

            # Update the model's token0 reference to point to the new token0 contract
            self.token0 = new_token0
            print(f"New {self.token0_symbol} token deployed at {self.token0.address} to ensure desired token order in the pool")

    def sync_pool_state(self):
        # Can add any other logic to sync pool with real pool
        if self.sync_pool_with_liq:
            tick_lower = price_to_valid_tick(self.initial_pool_price-self.initial_pool_price*0.5)
            tick_upper =price_to_valid_tick(self.initial_pool_price+self.initial_pool_price*0.5)
            self.add_liquidity(self.deployer, tick_lower, tick_upper, self.initial_liquidity_amount, b'')
            print(f'Initial liq amount {self.initial_liquidity_amount} added in pool')

        elif self.sync_pool_with_positions:
            for tick_idx, liquidity_gross, liquidity_net in self.state['ticks']:
                # Determine the tickLower and tickUpper based on tick_idx.
                # This is a simplified example; in a real scenario, you'd need to be more precise.
                tick_lower = int(tick_idx - 10)
                tick_upper = int(tick_idx + 10)

                # Calculate the amount of liquidity to add. 
                # This is a simplified example; you'd need to calculate this based on the pool's requirements.
                amount_liquidity = int((liquidity_gross + liquidity_net) / 2)

                # Call the contract method to add liquidity.
                # This is a placeholder; replace it with an actual contract interaction.
                self.pool.add_liquidity(GOD_ACCOUNT, tick_lower, tick_upper, amount_liquidity, '')

                print(f"Liquidity of {amount_liquidity} added between ticks {tick_lower} and {tick_upper}")

        elif self.sync_pool_with_ticks:
                for tick_idx, liquidity_gross, liquidity_net in self.state['ticks']:
                    # Determine the tickLower and tickUpper based on tick_idx.
                    # This is a simplified example; in a real scenario, you'd need to be more precise.
                    tick_lower = int(tick_idx - 10)
                    tick_upper = int(tick_idx + 10)

                    # Calculate the amount of liquidity to add. 
                    # This is a simplified example; you'd need to calculate this based on the pool's requirements.
                    amount_liquidity = int((liquidity_gross + liquidity_net) / 2)

                    # Call the contract method to add liquidity.
                    # This is a placeholder; replace it with an actual contract interaction.
                    self.pool.add_liquidity(GOD_ACCOUNT, tick_lower, tick_upper, amount_liquidity, '')

                    print(f"Liquidity of {amount_liquidity} added between ticks {tick_lower} and {tick_upper}")
        elif self.sync_pool_with_events:
            print("Computionally expensive process")
        else:
            print("No pool sync applied")


    def add_liquidity(self, liquidity_provider, tick_lower, tick_upper, usd_budget, data):
        tx_params = {'from': str(liquidity_provider), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        tx_params1 = {'from': str(GOD_ACCOUNT), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        tx_receipt=None
        try:
            pool_actions = self.pool
            liquidity=self.budget_to_liquidity(tick_lower,tick_upper,usd_budget)
            #print(liquidity)

            tx_receipt = pool_actions.mint(liquidity_provider, tick_lower, tick_upper, liquidity, data, tx_params)

            # Implement callback
            amount0 = tx_receipt.events['Mint']['amount0']
            amount1 = tx_receipt.events['Mint']['amount1']
            #print(tx_receipt.events['Mint']['amount'])
            print(tx_receipt.events)
            if amount0 > 0:
                tx_receipt_token0_transfer = self.token0.transfer(self.pool.address, amount0, tx_params)
            if amount1 > 0:
                tx_receipt_token1_transfer=self.token1.transfer(self.pool.address, amount1, tx_params)
                #print(f'token1 amount:{amount1}transfered to contract:{tx_receipt_token1_transfer}')

        except VirtualMachineError as e:
            print("Failed to add liquidty", e.revert_msg)

        #Transfer tokens token0 and token1 from GOD_ACCOUNT to agent's wallet(For safety instaed of this add acheck statement in policy which checks that agent's abalnce should be greater than amound he is adding in liquidty)
        #self.token0.transfer(liquidity_provider, amount0, tx_params1)
        #self.token1.transfer(liquidity_provider, amount1, tx_params1)

        # Store position in json file
        liquidity_provider_str = str(liquidity_provider)
        
        try:
            with open("model_storage/liq_positions.json", "r") as f:
                all_positions = json.load(f)
        except FileNotFoundError:
            all_positions = {}
        
        # Initialize if this pool_id is not in the list
        if self.pool_id not in all_positions:
            all_positions[self.pool_id] = {}
        
        # Initialize if this liquidity provider is not in the list
        if liquidity_provider_str not in all_positions[self.pool_id]:
            all_positions[self.pool_id][liquidity_provider_str] = []
        
        existing_position = None
        for position in all_positions[self.pool_id][liquidity_provider_str]:
            if position['tick_lower'] == tick_lower and position['tick_upper'] == tick_upper:
                existing_position = position
                break
    
        if existing_position:
            existing_position['liquidity'] += liquidity 
            existing_position['amount_usd'] += usd_budget # Add new liquidity to existing position
        else:
        # Add new position to list
            all_positions[self.pool_id][liquidity_provider_str].append({
                'tick_lower': tick_lower,
                'tick_upper': tick_upper,
                'liquidity': liquidity,
                'amount_usd':usd_budget
            })
        
        # Store updated positions
        with open("model_storage/liq_positions.json", "w") as f:
            json.dump(all_positions, f)
        
        return tx_receipt
    
    def add_liquidity_with_liquidity(self, liquidity_provider, tick_lower, tick_upper, liquidity, data):
        tx_params = {'from': str(liquidity_provider), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        tx_params1 = {'from': str(GOD_ACCOUNT), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        tx_receipt=None
        try:
            pool_actions = self.pool
            liquidity=liquidity

            tx_receipt = pool_actions.mint(liquidity_provider, tick_lower, tick_upper, liquidity, data, tx_params)

            # Implement callback
            amount0 = tx_receipt.events['Mint']['amount0']
            amount1 = tx_receipt.events['Mint']['amount1']
            print(tx_receipt.events)
            if amount0 > 0:
                tx_receipt_token0_transfer = self.token0.transfer(self.pool.address, amount0, tx_params)
            if amount1 > 0:
                tx_receipt_token1_transfer=self.token1.transfer(self.pool.address, amount1, tx_params)
                #print(f'token1 amount:{amount1}transfered to contract:{tx_receipt_token1_transfer}')


        except VirtualMachineError as e:
            print("Failed to add liquidty", e.revert_msg)

        #Transfer tokens token0 and token1 from GOD_ACCOUNT to agent's wallet(For safety instaed of this add acheck statement in policy which checks that agent's abalnce should be greater than amound he is adding in liquidty)
        #self.token0.transfer(liquidity_provider, amount0, tx_params1)
        #self.token1.transfer(liquidity_provider, amount1, tx_params1)

        # Store position in json file
        liquidity_provider_str = str(liquidity_provider)
        
        try:
            with open("model_storage/liq_positions.json", "r") as f:
                all_positions = json.load(f)
        except FileNotFoundError:
            all_positions = {}
        
        # Initialize if this pool_id is not in the list
        if self.pool_id not in all_positions:
            all_positions[self.pool_id] = {}
        
        # Initialize if this liquidity provider is not in the list
        if liquidity_provider_str not in all_positions[self.pool_id]:
            all_positions[self.pool_id][liquidity_provider_str] = []
        
        existing_position = None
        for position in all_positions[self.pool_id][liquidity_provider_str]:
            if position['tick_lower'] == tick_lower and position['tick_upper'] == tick_upper:
                existing_position = position
                break
    
        if existing_position:
            existing_position['liquidity'] += liquidity 
            existing_position['amount_usd'] += liquidity # Add new liquidity to existing position
        else:
        # Add new position to list
            all_positions[self.pool_id][liquidity_provider_str].append({
                'tick_lower': tick_lower,
                'tick_upper': tick_upper,
                'liquidity': liquidity,
                'amount_usd':liquidity
            })
        
        # Store updated positions
        with open("model_storage/liq_positions.json", "w") as f:
            json.dump(all_positions, f)
        
        return tx_receipt
    
    
    def remove_liquidity(self, liquidity_provider, tick_lower, tick_upper, amount_usd):
        liquidity_provider_str = str(liquidity_provider)
        tx_receipt = None
        
        # Convert budget to liquidity amount
        liquidity = self.budget_to_liquidity(tick_lower, tick_upper, amount_usd)

        try:
            tx_params = {'from': str(liquidity_provider), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
            tx_receipt = self.pool.burn(tick_lower, tick_upper, liquidity, tx_params)
            print(tx_receipt.events)
            self.collect_fee(liquidity_provider_str,tick_lower,tick_upper)
        except VirtualMachineError as e:
            print("Failed to remove liquidity", e.revert_msg)
            return tx_receipt  # Exit early if smart contract interaction fails

        try:
            with open("model_storage/liq_positions.json", "r") as f:
                all_positions = json.load(f)
        except FileNotFoundError:
            all_positions = {}
            
        if self.pool_id not in all_positions or \
        liquidity_provider_str not in all_positions[self.pool_id]:
            print("Position not found.")
            return tx_receipt  # Exit early if no positions are found

        existing_position = None
        for position in all_positions[self.pool_id][liquidity_provider_str]:
            if position['tick_lower'] == tick_lower and position['tick_upper'] == tick_upper:
                existing_position = position
                break

        if not existing_position:
            print("Position not found.")
            return tx_receipt  # Exit early if the specific position is not found

        if existing_position['liquidity'] > liquidity:
            existing_position['liquidity'] -= liquidity
            existing_position['amount_usd'] -= amount_usd  # Deduct removed liquidity
        else:
            all_positions[self.pool_id][liquidity_provider_str].remove(existing_position)  # Remove position if liquidity becomes zero
        
        # Update the JSON file
        with open("model_storage/liq_positions.json", "w") as f:
            json.dump(all_positions, f)

        return tx_receipt
    
    def remove_liquidity_with_liquidty(self, liquidity_provider, tick_lower, tick_upper, liquidity):
        liquidity_provider_str = str(liquidity_provider)
        tx_receipt = None
        
        # Convert budget to liquidity amount

        try:
            tx_params = {'from': str(liquidity_provider), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
            tx_receipt = self.pool.burn(tick_lower, tick_upper, liquidity, tx_params)
            print(tx_receipt.events)
            #self.collect_fee(liquidity_provider_str,tick_lower,tick_upper)
        except VirtualMachineError as e:
            print("Failed to remove liquidity", e.revert_msg)
            return tx_receipt  # Exit early if smart contract interaction fails

        try:
            with open("model_storage/liq_positions.json", "r") as f:
                all_positions = json.load(f)
        except FileNotFoundError:
            all_positions = {}
            
        if self.pool_id not in all_positions or \
        liquidity_provider_str not in all_positions[self.pool_id]:
            print("Position not found.")
            return tx_receipt  # Exit early if no positions are found

        existing_position = None
        for position in all_positions[self.pool_id][liquidity_provider_str]:
            if position['tick_lower'] == tick_lower and position['tick_upper'] == tick_upper:
                existing_position = position
                break

        if not existing_position:
            print("Position not found.")
            return tx_receipt  # Exit early if the specific position is not found

        if existing_position['liquidity'] > liquidity:
            existing_position['liquidity'] -= liquidity
            existing_position['amount_usd'] -= liquidity  # Deduct removed liquidity
        else:
            all_positions[self.pool_id][liquidity_provider_str].remove(existing_position)  # Remove position if liquidity becomes zero
        
        # Update the JSON file
        with open("model_storage/liq_positions.json", "w") as f:
            json.dump(all_positions, f)

        return tx_receipt
            

    def swap_token0_for_token1(self, recipient, amount_specified, data):
        tx_params = {'from': str(recipient), 'gas_price': self.base_fee + 1000000, 'gas_limit':  5000000, 'allow_revert': True}
        #tx_params1={'from': str(GOD_ACCOUNT), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        sqrt_price_limit_x96=4295128740+1

        pool_actions = self.pool
        zero_for_one = True
        tx_receipt=None
        
        try:
            tx_receipt= pool_actions.swap(recipient, zero_for_one, amount_specified,sqrt_price_limit_x96, data,tx_params)
            
            print(tx_receipt.events)
            amount0 = tx_receipt.events['Swap']['amount0']

            #Transfer tokens from GOD_ACCOUNT to agent's wallet
            #self.token0.transfer(recipient, amount0, tx_params1)

            # Transfer token0 to pool (callback)
            tx_receipt_token0_transfer = self.token0.transfer(self.pool.address, amount0, tx_params)
            
        
        except VirtualMachineError as e:
            print("Swap token 0 to Token 1 Transaction failed:", e.revert_msg)
            slot0_data = self.pool.slot0()
            print(f'contract_token1_balance - approx_token1_amount: {self.token1.balanceOf(self.pool)-amount_specified*sqrtp_to_price(slot0_data[0])}, approx_token1_amount: {amount_specified*sqrtp_to_price(slot0_data[0])}), contract_token1_balance: {self.token1.balanceOf(self.pool)}, amount_swap_token0: {amount_specified}, contract_token0 _balance - amount_swap_token0: {self.token0.balanceOf(self.pool)-amount_specified}')

        return tx_receipt
     
    def swap_token1_for_token0(self, recipient, amount_specified, data):
        tx_params = {'from': str(recipient), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        tx_params1={'from': str(GOD_ACCOUNT), 'gas_price': self.base_fee + 1, 'gas_limit': 5000000, 'allow_revert': True}
        sqrt_price_limit_x96=1461446703485210103287273052203988822378723970342-1

        pool_actions = self.pool   
        zero_for_one = False
        tx_receipt=None 

        try:
            tx_receipt = pool_actions.swap(recipient, zero_for_one, amount_specified, sqrt_price_limit_x96, data,tx_params)
            print(tx_receipt.events)
        
            amount1 = tx_receipt.events['Swap']['amount1']

            #Transfer tokens token0 and token1 from GOD_ACCOUNT to agent's wallet (This should be removed latter as our agent will have balace in their accounts while initialized and they should not be allowed to make a transaction greater than their balance which will result in failure of transaction)
            #self.token1.transfer(recipient, amount1, tx_params1)

            # Trasfer token1 to pool (callabck)
            tx_receipt_token1_transfer = self.token1.transfer(self.pool.address, amount1, tx_params)
            #print(f'token1 amount:{amount1} transfered to contract:{tx_receipt_token1_transfer}')
            
        
        except VirtualMachineError as e:
            print("Swap token 1 to Token 0 Transaction failed:", e.revert_msg)
            slot0_data = self.pool.slot0()
            print(f'contract_token0_balance - approx_token0_amount: {self.token0.balanceOf(self.pool)-amount_specified/sqrtp_to_price(slot0_data[0])}, approx_token0_amount: {amount_specified/sqrtp_to_price(slot0_data[0])}, contract_token0_balance: {self.token0.balanceOf(self.pool)}, contract_token1_balance - amount_swap_token1: {self.token1.balanceOf(self.pool)-amount_specified}')
        return tx_receipt

    def collect_fee(self,recipient,tick_lower,tick_upper):
        tx_params = {'from': str(recipient), 'gas_price': self.base_fee + 1, 'gas_limit': 500000000, 'allow_revert': True}
        # Poke to update variables
        try:
            tx_receipt = self.pool.burn(tick_lower, tick_upper, 0, tx_params)
        except VirtualMachineError as e:
            print("Poke:", e.revert_msg)
        
        position_key = Web3.solidityKeccak(['address', 'int24', 'int24'], [str(recipient), tick_lower, tick_upper]).hex()

        position_info = self.pool.positions(position_key)
        
        amount0Owed = position_info[3]
        amount1Owed = position_info[4]

        print(f'amount0Owed: {position_info[3]}, ,amount1Owed: {position_info[4]}')

        tx_receipt=None
        fee_collected_usd=0
        try:
            tx_receipt=self.pool.collect(recipient,tick_lower,tick_upper,amount0Owed, amount1Owed,tx_params)
            print(tx_receipt.events)

            amount0Collected=tx_receipt.events['Collect']['amount0']
            amount1Collected=tx_receipt.events['Collect']['amount1']

            slot0_data = self.pool.slot0()
            fee_collected_usd = fromBase18(amount0Collected*sqrtp_to_price(slot0_data[0]) + amount1Collected)
        except VirtualMachineError as e:
            print("Fee collection failed:", e.revert_msg)
            print(f"contract_token0_balance - amount0Owed: {self.token0.balanceOf(self.pool)-amount0Owed} ,contract_token1_balance - amount1Owed: {self.token1.balanceOf(self.pool)-amount1Owed}, position_tick_lower: {tick_lower}, position_tick_upper: {tick_upper}")

        
        #print(f"Fee collected usd: {fee_collected_usd}")
        return tx_receipt,fee_collected_usd
    
   
    # Get All positions of all LPs in the pool
    def get_all_liquidity_positions(self):
        try:
            with open("model_storage/liq_positions.json", "r") as f:
                all_positions = json.load(f)
        except FileNotFoundError:
            print("No positions found.")
            return {}
        except json.JSONDecodeError:
            print("Error decoding JSON. File might be malformed.")
            return {}

        if self.pool_id in all_positions:
            return all_positions[self.pool_id]
        else:
            print(f"No positions found for pool {self.pool_id}.")
            return {}

    # Get all positions of an LP in the pool
    def get_lp_all_positions(self, liquidity_provider):
        liquidity_provider_str = str(liquidity_provider)
        all_positions = self.get_all_liquidity_positions()

        if not all_positions:
            print("Pool has no LP positions.")
            return None

        if liquidity_provider_str in all_positions:
            return all_positions[liquidity_provider_str]
        else:
            print(f"No positions found for this liquidity provider {liquidity_provider_str} in pool.")
            return None

    def get_position_state(self,tick_lower,tick_upper,agent):
        position_key = Web3.solidityKeccak(['address', 'int24', 'int24'], [str(agent), tick_lower, tick_upper]).hex()
        pool_state = self.pool
        position_info = self.pool.positions(position_key)
        
        return {
        "position_key":f"{str(agent)}_{tick_lower}_{tick_upper}",
        "liquidity_provider": str(agent),
        "tick_lower":tick_lower,
        "tick_upper":tick_upper,
        "_liquidity_raw": position_info[0],
        #"_liquidity_converted": fromBase18(position_info[0]),
        "feeGrowthInside0LastX128": position_info[1],
        #"feeGrowthInside0Last": fromBase128(position_info[1]),
        "feeGrowthInside1LastX128": position_info[2],
        #"feeGrowthInside1Last": fromBase128(position_info[2]),
        "tokensOwed0_raw": position_info[3],
        #"tokensOwed0_converted": fromBase18(position_info[3]),
        "tokensOwed1_raw": position_info[4],
        #"tokensOwed1_converted": fromBase18(position_info[4])
    }


    def get_tick_state(self,tick):
        pool_state = self.pool
        word_position = tick >> 8

        tick_info = pool_state.ticks(tick)
        tick_bitmap = pool_state.tickBitmap(word_position)

        return {
        'tick':tick,
        "liquidityGross_raw": tick_info[0],
        #"liquidityGross_converted": fromBase18(tick_info[0]),
        "liquidityNet_raw": tick_info[1],
        #"liquidityNet_converted": fromBase18(tick_info[1]),
        "feeGrowthOutside0X128": tick_info[2],
        #"feeGrowthOutside0": fromBase128(tick_info[2]),
        "feeGrowthOutside1X128": tick_info[3],
        #"feeGrowthOutside1": fromBase128(tick_info[3]),
        "tickCumulativeOutside": tick_info[4],
        #"secondsPerLiquidityOutsideX128": tick_info[5],
        #"secondsPerLiquidityOutside": fromBase128(tick_info[5]),
        #"secondsOutside": tick_info[6],
        #"initialized": tick_info[7],
        "tickBitmap": tick_bitmap
    }
    
    def get_global_state(self):
        pool_state = self.pool
        slot0_data = pool_state.slot0()
        observation_index = slot0_data[2]

        feeGrowthGlobal0X128 = pool_state.feeGrowthGlobal0X128()
        feeGrowthGlobal1X128 = pool_state.feeGrowthGlobal1X128()
        protocol_fees = pool_state.protocolFees()
        
        liquidity = pool_state.liquidity()

        observation_info = pool_state.observations(observation_index)
        
        return {
        "curr_sqrtPriceX96": slot0_data[0],
        "curr_price": sqrtp_to_price(slot0_data[0]),
        "tick": slot0_data[1],
        #"locking_status": slot0_data[6],
        "feeGrowthGlobal0X128": feeGrowthGlobal0X128,
        #"feeGrowthGlobal0": fromBase128(feeGrowthGlobal0X128),
        "feeGrowthGlobal1X128": feeGrowthGlobal1X128,
        #"feeGrowthGlobal1": fromBase128(feeGrowthGlobal1X128),
        "liquidity_raw": liquidity,
        #"liquidity_converted": fromBase18(liquidity),
        "blockTimestamp": observation_info[0],
        "tickCumulative": observation_info[1],
        "secondsPerLiquidityCumulativeX128": observation_info[2],
        #"secondsPerLiquidityCumulative": fromBase128(observation_info[2]),
        #"initialized": observation_info[3]
    }
           
    def get_pool_state_for_all_ticks(self, lower_price_interested, upper_price_interested):
        tick_states = {} 
        try:
            with open("model_storage/liq_positions.json", "r") as f:
                file_content = f.read().strip()  # Read and remove any leading/trailing whitespace
                if not file_content:  # Check if file is empty
                    print("File is empty. Returning.")
                    return tick_states  # Return empty dict
                all_positions = json.loads(file_content)
        except FileNotFoundError:
            print("No positions found.")
            return tick_states  # Return empty dict
        except json.JSONDecodeError:
            print("Error decoding JSON. File might be malformed.")
            return tick_states  # Return empty dict

        # Convert interested prices to ticks
        lower_tick_interested = price_to_valid_tick(lower_price_interested, tick_spacing=60)
        upper_tick_interested = price_to_valid_tick(upper_price_interested, tick_spacing=60)

        unique_ticks = set()

        # Filter only the positions related to this specific pool.
        if self.pool_id not in all_positions:
            print("No positions for this pool.")
            return tick_states

        for liquidity_provider, positions in all_positions[self.pool_id].items():
            for position in positions:
                tick_lower = position['tick_lower']
                tick_upper = position['tick_upper']

                # Check if the tick_lower or tick_upper falls within the interested range
                if lower_tick_interested <= tick_lower <= upper_tick_interested or \
                lower_tick_interested <= tick_upper <= upper_tick_interested:
                    unique_ticks.add(tick_lower)
                    unique_ticks.add(tick_upper)

        # Fetch pool states for unique ticks within the range
        for tick in unique_ticks:
            tick_states[tick] = self.get_tick_state(tick)  # Fetch and store each tick state

        return tick_states

    
    def get_pool_state_for_all_positions(self):
        position_states = {}
        # Load all positions from the JSON file
        try:
            with open("model_storage/liq_positions.json", "r") as f:
                all_positions = json.load(f)
        except FileNotFoundError:
            print("No positions found.")
            return

        # Check if this pool_id exists in all_positions
        if self.pool_id not in all_positions:
            print(f"No positions found for pool {self.pool_id}.")
            return

        # Fetch positions for this specific pool
        for liquidity_provider_str, positions in all_positions[self.pool_id].items():
            for position in positions:
                tick_lower = position['tick_lower']
                tick_upper = position['tick_upper']
                liquidity = position['liquidity']
                position_key = f"{liquidity_provider_str}_{tick_lower}_{tick_upper}"
                position_states[position_key] = self.get_position_state(tick_lower, tick_upper,liquidity_provider_str)

        return position_states

        

    def get_wallet_balances(self, recipient):
        recipient_address = recipient.address  # Assuming recipient is a brownie account object
        balances = {
        recipient_address: {
            'ETH': fromBase18(recipient.balance()),
            'token0': fromBase18(self.token0.balanceOf(recipient_address)),
            'token1': fromBase18(self.token1.balanceOf(recipient_address))
        }
    }
        return balances    
        

    def set_pool_allowance(self, recipient, amount0,amount1):
        '''
        w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
        base_fee = w3.eth.getBlock('latest')['baseFeePerGas']
        '''
        tx_params = {'from': str(recipient),'gas_price': self.base_fee + 1}
        
        tx_receipt_token0 = self.token0.approve(self.pool.address, amount0, tx_params)
           
        print(tx_receipt_token0.events)
            
        
        
        tx_receipt_token1 = self.token1.approve(self.pool.address, amount1, tx_params)
            
        print(tx_receipt_token1.events)
    
   
    def budget_to_liquidity(self,tick_lower,tick_upper,usd_budget):
            
        # Calculating liquidity (Not needed here: Can shift this function to helper_functions)
        def get_liquidity_for_amounts(sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, amount0, amount1):
            if sqrt_ratio_a_x96 > sqrt_ratio_b_x96:
                sqrt_ratio_a_x96, sqrt_ratio_b_x96 = sqrt_ratio_b_x96, sqrt_ratio_a_x96
            
            if sqrt_ratio_x96 <= sqrt_ratio_a_x96:
                return liquidity0(amount0, sqrt_ratio_a_x96, sqrt_ratio_b_x96)
            elif sqrt_ratio_x96 < sqrt_ratio_b_x96:
                liquidity0_value = liquidity0(amount0, sqrt_ratio_x96, sqrt_ratio_b_x96)
                liquidity1_value = liquidity1(amount1, sqrt_ratio_a_x96, sqrt_ratio_x96)
                return min(liquidity0_value, liquidity1_value)
            else:
                return liquidity1(amount1, sqrt_ratio_a_x96, sqrt_ratio_b_x96)

        slot0_data = self.pool.slot0()
        sqrtp_cur =slot0_data[0]

        usdp_cur = sqrtp_to_price(sqrtp_cur)
        amount_token0 =  ((0.5 * usd_budget)/usdp_cur) * eth
        amount_token1 = 0.5 * usd_budget * eth

        sqrtp_low = tick_to_sqrtp(tick_lower)
        sqrtp_upp = tick_to_sqrtp(tick_upper)
        
        
        liquidity=get_liquidity_for_amounts(sqrt_ratio_x96=sqrtp_cur, sqrt_ratio_a_x96=sqrtp_low, sqrt_ratio_b_x96=sqrtp_upp, amount0=amount_token0, amount1=amount_token1)
        return liquidity

'''
Usage
token0 = "ETH"
token1 = "DAI"
supply_token0 = 1e18
supply_token1 = 1e18
decimals_token0 = 18
decimals_token1 = 18
fee_tier = 3000
initial_pool_price = 2000
deployer = GOD_ACCOUNT
sync_pool=True
initial_liquidity_amount=10000
env = UniV3Model(token0, token1,decimals_token0,decimals_token1,supply_token0,supply_token1,fee_tier,initial_pool_price,deployer,sync_pool, initial_liquidity_amount)
'''


'\nUsage\ntoken0 = "ETH"\ntoken1 = "DAI"\nsupply_token0 = 1e18\nsupply_token1 = 1e18\ndecimals_token0 = 18\ndecimals_token1 = 18\nfee_tier = 3000\ninitial_pool_price = 2000\ndeployer = GOD_ACCOUNT\nsync_pool=True\ninitial_liquidity_amount=10000\nenv = UniV3Model(token0, token1,decimals_token0,decimals_token1,supply_token0,supply_token1,fee_tier,initial_pool_price,deployer,sync_pool, initial_liquidity_amount)\n'

In [ ]:
token0 = "wETH"
token1 = "DAI"
supply_token0 = 1e18
supply_token1 = 1e18
decimals_token0 = 18
decimals_token1 = 18
fee_tier = 3000
initial_pool_price = 2000
deployer = GOD_ACCOUNT
sync_pool=True
initial_liquidity_amount=100
deployed_pool = UniV3Model(token0, token1,decimals_token0,decimals_token1,supply_token0,supply_token1,fee_tier,initial_pool_price,deployer,sync_pool, initial_liquidity_amount)

Existing pool:wETH_DAI_3000 having pool address: 0x58Ce5B9f4e3d0222bbAB5904d914EB069dE6adDb loaded


In [ ]:
deployed_pool.token0.address

'0x3F199996b3CC56AAE18e04e544257A17D5574be8'

In [ ]:
deployed_pool.pool.token0()

'0x3F199996b3CC56AAE18e04e544257A17D5574be8'

In [ ]:
from util.globaltokens import weth_usdc_pool
from util.constants import GOD_ACCOUNT
from util.base18 import toBase18, price_to_valid_tick
env=weth_usdc_pool
print(env.get_lp_all_positions(GOD_ACCOUNT))

In [ ]:

global_state=env.get_global_state()
global_state_df=pd.DataFrame([global_state])
global_state_df

,curr_sqrtPriceX96,curr_price,tick,feeGrowthGlobal0X128,feeGrowthGlobal1X128,liquidity_raw,blockTimestamp,tickCumulative,secondsPerLiquidityCumulativeX128
0,3541946836890595876327551269414,1998.595518,76005,8043322633717853653763433794043,0,380755745076068400000,1697394631,34712170838,408135170782626187638558


In [ ]:
tick_state=env.get_pool_state_for_all_ticks(1000,3000)
tick_state_df = pd.DataFrame.from_dict(tick_state, orient='index')
tick_state_df

,tick,liquidityGross_raw,liquidityNet_raw,feeGrowthOutside0X128,feeGrowthOutside1X128,tickCumulativeOutside,tickBitmap
74400,74400,0,0,0,0,0,0
69060,69060,380755745076068400000,380755745076068400000,0,0,0,0
78660,78660,21470565349021557000000,21470565349021557000000,0,0,0,0
80040,80040,380755745076068400000,-380755745076068400000,0,0,0,0
79020,79020,21470565349021557000000,-21470565349021557000000,0,0,0,0
76980,76980,0,0,0,0,0,0
77400,77400,0,0,0,0,0,0
74940,74940,0,0,0,0,0,0


In [ ]:
positions_data=env.get_pool_state_for_all_positions()
positions_df = pd.DataFrame.from_dict(positions_data, orient='index')
positions_df

,position_key,liquidity_provider,tick_lower,tick_upper,_liquidity_raw,feeGrowthInside0LastX128,feeGrowthInside1LastX128,tokensOwed0_raw,tokensOwed1_raw
0x330997E70b83f1a562490FCaA5996314fA5a971a_74940_76980,0x330997E70b83f1a562490FCaA5996314fA5a971a_749...,0x330997E70b83f1a562490FCaA5996314fA5a971a,74940,76980,0,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_74400_77400,0x330997E70b83f1a562490FCaA5996314fA5a971a_744...,0x330997E70b83f1a562490FCaA5996314fA5a971a,74400,77400,0,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_69060_80040,0x330997E70b83f1a562490FCaA5996314fA5a971a_690...,0x330997E70b83f1a562490FCaA5996314fA5a971a,69060,80040,380755745076068400000,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_7600_8100,0x330997E70b83f1a562490FCaA5996314fA5a971a_760...,0x330997E70b83f1a562490FCaA5996314fA5a971a,7600,8100,0,0,0,0,0
0x330997E70b83f1a562490FCaA5996314fA5a971a_78660_79020,0x330997E70b83f1a562490FCaA5996314fA5a971a_786...,0x330997E70b83f1a562490FCaA5996314fA5a971a,78660,79020,21470565349021557000000,0,0,0,0


# Test model, Simulation and agent env

In [ ]:
def test_env(episodes,env):
    #Test environemnt
    episodes = episodes
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
        
        while not done:
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action)
            score+=reward
        print('Episode:{} Score:{}'.format(episode, score))
    env.close()

In [ ]:
import subprocess

cmd = "rm -rf model_outdir_csv; tsp run netlists/uniswapV3/netlist.py model_outdir_csv"
result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print(f'Standard Output:\n{result.stdout}')
print(f'Standard Error:\n{result.stderr}')

In [ ]:
from netlists.uniswapV3.netlist import SimStrategy,SimState,netlist_createLogData
from util.globaltokens import weth_usdc_pool,eth_dai_pool,btc_usdt_pool

sim_strategy = SimStrategy()
sim_state = SimState(ss=sim_strategy,pool=weth_usdc_pool)

output_dir = "model_outdir_csv"
netlist_log_func = netlist_createLogData

from engine.SimEngine import SimEngine
engine = SimEngine(sim_state, output_dir, netlist_log_func)

retail_lp_agent=sim_state.agents['retail_lp']._wallet.address
print(f'retail_lp_agent: {retail_lp_agent}')

noise_trader=sim_state.agents['noise_trader']._wallet.address
print(f'noise_trader_agent: {noise_trader}')

mnemonic: 'meat net all market night spoon shoe come cable bid combine tip'
funded account with token0: {'Transfer': [OrderedDict([('from', '0x330997E70b83f1a562490FCaA5996314fA5a971a'), ('to', '0x479894340099e5b3a09C9EE31A252EA53f62dDFd'), ('value', 1000000000000000019884624838656)])]}
funded account with token1: {'Transfer': [OrderedDict([('from', '0x330997E70b83f1a562490FCaA5996314fA5a971a'), ('to', '0x479894340099e5b3a09C9EE31A252EA53f62dDFd'), ('value', 110000000000000000498458871988224)])]}
mnemonic: 'gate evoke disorder detail stem helmet choice wash song yellow magic forward'
funded account with token0: {'Transfer': [OrderedDict([('from', '0x330997E70b83f1a562490FCaA5996314fA5a971a'), ('to', '0xb889D710d487C9AB5b6A0f8Bc73B0e142fE410fd'), ('value', 4999999999999999727876154935214080)])]}
funded account with token1: {'Transfer': [OrderedDict([('from', '0x330997E70b83f1a562490FCaA5996314fA5a971a'), ('to', '0xb889D710d487C9AB5b6A0f8Bc73B0e142fE410fd'), ('value', 5500000000000000565

## Custom Envs based on fully defined action and obs space

# Stablebaseline agents

In [ ]:
def preprocessing_data():
  # Instead of getting data from csv file get this data directly from pool using pool's get_position, get_tick and global state functions 
    
    data_df = pd.read_csv('/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice/outdir_csv/data.csv')
    events_log_df = pd.read_csv('/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice/outdir_csv/events_log.csv')
    position_data_df = pd.read_csv('/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice/outdir_csv/position_data.csv')
    tick_data_df = pd.read_csv('/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice/outdir_csv/tick_data.csv')


    #global_state_data = data_df.drop(columns=['Tick', 'Second', 'Min', 'Hour', 'Day'])
    global_state_data = data_df.iloc[-1:]
    
    global_state_data.astype(float)
    
    # Filter events_log.csv to get only the "Swap" events and calculate net data of swaps
    swap_events = events_log_df[events_log_df['Event Type'] == 'Swap']
    if 'amount1' in swap_events.columns:
        swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')
        net_swaps = swap_events['amount1'].abs().sum()
    else:
        net_swaps = 0
    # Drop unnecessary columns for position_data and tick_data
    df_position_clean = position_data_df.drop(columns=['Unnamed: 0','Unnamed: 1','liquidity_provider','position_key'])
    df_tick_clean = tick_data_df.drop(columns=['Unnamed: 0','Unnamed: 1','tickBitmap'])

    # Convert columns to appropriate numeric types for position_data
    cols_to_convert_position = [
        '_liquidity_raw', 
        'feeGrowthInside0LastX128', 
        'feeGrowthInside1LastX128', 
        'tokensOwed0_raw', 
        'tokensOwed1_raw'
    ]
    df_position_clean[cols_to_convert_position] = df_position_clean[cols_to_convert_position].apply(pd.to_numeric, errors='coerce')

    # Convert columns to appropriate numeric types for tick_data
    cols_to_convert_tick = [
        'liquidityGross_raw', 
        'liquidityNet_raw', 
        'feeGrowthOutside0X128', 
        'feeGrowthOutside1X128'
    ]
    df_tick_clean[cols_to_convert_tick] = df_tick_clean[cols_to_convert_tick].apply(pd.to_numeric, errors='coerce')

    # Handle any NaN values by filling them with zeros
    df_position_clean = df_position_clean.fillna(0)
    df_tick_clean = df_tick_clean.fillna(0)

    # Normalize all the features
    #scaler = MinMaxScaler()
    #df_position_normalized = pd.DataFrame(scaler.fit_transform(df_position_clean.drop(columns=['Unnamed: 1', 'liquidity_provider'])), columns=df_position_clean.drop(columns=['Unnamed: 1', 'liquidity_provider']).columns)
    #df_tick_normalized = pd.DataFrame(scaler.fit_transform(df_tick_clean), columns=df_tick_clean.columns)

    return df_position_clean, df_tick_clean, global_state_data,net_swaps

df_position_normalized, df_tick_normalized,global_state_data,net_swaps=preprocessing_data()
position_columns=df_position_normalized.columns
tick_columns=df_tick_normalized.columns

/tmp/ipykernel_20290/3389155064.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')


In [ ]:
global_state_data.head()
df_position_normalized.head()

,tick_lower,tick_upper,_liquidity_raw,feeGrowthInside0LastX128,feeGrowthInside1LastX128,tokensOwed0_raw,tokensOwed1_raw
0,75480,76500,6.379481e+21,0.0,0.0,0,0
1,75240,76980,4.736852e+21,0.0,0.0,0,0
2,74940,77400,4.171466e+21,0.0,0.0,0,0
3,74700,77820,3.882399e+21,0.0,0.0,0,0
4,74400,78240,3.713437e+21,0.0,0.0,0,0


In [ ]:
import pandas as pd
import numpy as np
from gym.spaces import Dict, Box

def create_obs_space():
    position_data_df,tick_data_df,global_state_data,net_swaps= preprocessing_data()
    # Load the CSV files
    
    # Prepare position_data.csv and tick_data.csv
    position_data_dict = position_data_df.to_dict(orient='list')
    tick_data_dict = tick_data_df.to_dict(orient='list')
    global_state_data_dict = global_state_data.to_dict(orient='list')

    
    position_data_dict = {k.strip(): v for k, v in position_data_dict.items()}
    tick_data_dict = {k.strip(): v for k, v in tick_data_dict.items()}
    global_state_data = {k.strip(): (v.strip() if isinstance(v, str) else v) for k, v in global_state_data_dict.items()}
    
    # Create the Gym observation space
    observation_space = Dict({
        'global_state': Dict({key: Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32) for key in global_state_data.keys()}),
        'net_swaps': Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
        'position_state': Dict({key: Box(low=-np.inf, high=np.inf, shape=(len(value),), dtype=np.float32) for key, value in position_data_dict.items()}),
        'tick_state': Dict({key: Box(low=-np.inf, high=np.inf, shape=(len(value),), dtype=np.float32) for key, value in tick_data_dict.items()})
    })
    
    return observation_space

In [ ]:
observation_space=create_obs_space()
#observation_space.sample()

/tmp/ipykernel_20290/3389155064.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')


In [ ]:
#Should pass lp_positions of agent as input so that it can select position to reomve from those active positions
class CustomActionSpace(Space):
    def __init__(self, min_tick, max_tick, lp_budget):
        super().__init__(shape=(1,), dtype=object)
        self.min_tick = min_tick
        self.max_tick = max_tick
        self.lp_budget = lp_budget
        self.action_types = ['do_nothing', 'add_liq', 'remove_liq']

    def sample(self):
        action_type = np.random.choice(self.action_types)

        if action_type == 'do_nothing':
            return {'action_type': 'do_nothing'}
        
        elif action_type == 'add_liq':
            tick_lower = np.random.randint(self.min_tick, self.max_tick)
            tick_upper = np.random.randint(tick_lower, self.max_tick)
            amount_usd = np.random.randint(0, self.lp_budget)
            return {'action_type': 'add_liq', 'amount_usd': amount_usd, 'tick_lower': tick_lower, 'tick_upper': tick_upper}

        elif action_type == 'remove_liq':
            tick_lower = np.random.randint(self.min_tick, self.max_tick)
            tick_upper = np.random.randint(tick_lower, self.max_tick)
            amount_liq = np.random.randint(0, self.lp_budget)
            return {'action_type': 'remove_liq', 'amount_liq': amount_liq, 'tick_lower': tick_lower, 'tick_upper': tick_upper}

    def contains(self, x):
        #Add condition for reomve liq so that reomve liq position should not be an invalid position
        if not isinstance(x, dict):
            return False
        if 'action_type' not in x:
            return False

        action_type = x['action_type']

        if action_type == 'do_nothing':
            return True
        elif action_type in ['add_liq', 'remove_liq']:
            if all(key in x for key in ['amount_usd', 'tick_lower', 'tick_upper']):
                return (self.min_tick <= x['tick_lower'] < x['tick_upper'] <= self.max_tick) and \
                    (0 <= x['amount_usd'] <= self.lp_budget)
        return False


# Test the custom action space
min_tick = -88705
max_tick = -min_tick
lp_budget = 50

action_space = CustomActionSpace(min_tick, max_tick, lp_budget)

In [ ]:
action_space.sample()

{'action_type': 'add_liq',
 'amount_usd': 9,
 'tick_lower': 54249,
 'tick_upper': 77298}

## Custom Envs based on fully defined action and obs space

In [ ]:
class CustomEnv(gym.Env):
    def __init__(self):
        super(CustomEnv, self).__init__()

        # Action space
        self.action_space = CustomActionSpace(min_tick, max_tick, lp_budget)
        self.observation_space = create_obs_space()
        
        self.reset()
    def step(self, action):
        obs = self.get_obs_space()
        action_type=action['action_type']

        if action_type=='do_nothing':
            print('rl agent hloding position')

        elif action_type=='add_liq':
            tick_lower=action['tick_lower']
            tick_upper=action['tick_upper']
            amount=action['amount_usd']
            self.pool.add_liquidity(GOD_ACCOUNT, tick_lower, tick_upper, amount, b'')

        elif action_type=='remov_liq':
            tick_lower=action['tick_lower']
            tick_upper=action['tick_upper']
            amount_liq=action['amount_liq']
            weth_usdc_pool.remove_liquidity(tick_lower,tick_upper,amount_liq)

        
        #reward = calculate_reward() # Calculate reward
        done = False  # Check if episode is done
        reward=10
        info={}
        return obs, done,reward,info

    def reset(self):
        df_position_clean, df_tick_clean, global_state_data, net_swaps = preprocessing_data()
        
        # Use the data to populate the initial state
        initial_state = {
            'position_state': {col: df_position_clean[col].values for col in df_position_clean.columns},
            'tick_state': {col: df_tick_clean[col].values for col in df_tick_clean.columns},
            'global_state': {col: global_state_data[col].values[0] if global_state_data[col].values.size > 0 else np.nan for col in global_state_data.columns},
            'net_swaps': np.array([net_swaps])
          } 
        return initial_state
    
    def get_obs_space(self):
    # Similar to your preprocessing_data() but designed to get the current state
        df_position_clean, df_tick_clean, global_state_data, net_swaps = preprocessing_data()
        
        # Populate the state using the real data
        current_state = {
            'position_state': {col: df_position_clean[col].values for col in df_position_clean.columns},
            'tick_state': {col: df_tick_clean[col].values for col in df_tick_clean.columns},
            'global_state': {col: global_state_data[col].values[0] if global_state_data[col].values.size > 0 else np.nan for col in global_state_data.columns},
            'net_swaps': np.array([net_swaps])
        }
        return current_state
    def calculate_reward():
        pass


In [ ]:
env=CustomEnv()

/tmp/ipykernel_20290/3389155064.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')
/tmp/ipykernel_20290/3389155064.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')


In [ ]:
#Test environemnt
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

/tmp/ipykernel_20290/3389155064.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')
/tmp/ipykernel_20290/3389155064.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swap_events['amount1'] = pd.to_numeric(swap_events['amount1'], errors='coerce')

KeyboardInterrupt



## Discrete Env

In [ ]:
import gym
from gym import spaces

class DiscreteEnv(gym.Env):
    def __init__(self, df_position_normalized, df_tick_normalized):
        super(CustomEnv, self).__init__()

        self.df_position = df_position_normalized.values  # Convert to numpy array
        self.df_tick = df_tick_normalized.values 
        
        
        self.action_space=Discrete(3)
        self.reward=0
        self.observation_space = gym.spaces.Dict({
            'position_data': gym.spaces.Box(low=-np.inf, high=np.inf, shape=self.df_position.shape, dtype=np.float32),
            'tick_data': gym.spaces.Box(low=-np.inf, high=np.inf, shape=self.df_tick.shape, dtype=np.float32),
            'curr_price': gym.spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            'curr_liq': gym.spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            'volume_usd': gym.spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            'price_volatility': gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        })

    def reset(self):
        # Reset the state of the environment to an initial state
        self.state = self.observation_space.sample()
        return self.state

    def step(self, action):
        #These dfs should have updated data of pool at each step 
        obs = {
            'position_data': self.df_position,  # Replace with appropriate data for the current step
            'tick_data': self.df_tick,  # Replace with appropriate data for the current step
            'curr_price': np.array([1000.0]),  # Placeholder
            'curr_liq': np.array([2000.0]),  # Placeholder
            'volume_usd': np.array([3000.0]),  # Placeholder
            'price_volatility': np.array([0.05])  # Placeholder
        }
        # Execute agent's action using pool's interface of add/remove liquidity
        self._take_action(action)
        
        # run uniswap abm env of n_steps
        # tsp run netlists  and get new state files() using preprocessing data function
        self.reward += self._calculate_reward()

        self.state=obs

        # Assume the episode is done after some arbitrary condition
        done = self._is_done()
        
        return self.state, self.reward, done, {}

    def _take_action(self, action):
        action_type = action
        
        if action_type == 0:
            # Do not change the state
            pass
        
        elif action_type == 1:
            tick_lower = 1400#action['tick_lower']
            tick_upper = 2400#action['tick_upper']
            amount = 2000#action['amount_usd']
            add_liq=[1900,2000,1200]
            
            self.pool.add_liquidity(GOD_ACCOUNT, tick_lower, tick_upper, amount, b'')
        
        elif action_type == 2:
            tick_lower = 1400#action['tick_lower']
            tick_upper = 2400#action['tick_upper']
            amount = 2000#action['amount_usd']
            remove_liq=[1900,2000,1200]
            
            self.pool.remove_liquidity(GOD_ACCOUNT, tick_lower, tick_upper, amount)


    def _calculate_reward(self):
        # get fee collected by agent 
        # self.pool.collect()
        # from pool value from previous state to this state get IL
        return 10

    def _is_done(self):
        # Implement your episode termination logic here
        
        if self.reward>50:
            return True
        else:
            self.reward+=10
            return False

# Implement your CustomActionSpace class here


In [ ]:
env = CustomEnv(df_position_normalized,df_tick_normalized)

In [ ]:
#Test environemnt
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Transaction sent: 0xa8658ffeb5c6787bfd1c9ae9dab07fdcfda58740116f7fb8e7aa037b6004ecb6
  Gas price: 8e-09 gwei   Gas limit: 5000000   Nonce: 541
  UniswapV3Pool.burn confirmed (LS)   Block: 674   Gas used: 40630 (0.81%)

Failed to remove liquidty LS
Transaction sent: 0xecec82ebbc5d2c7a00838308703b124e748c06c3dd307f423d2eebf337824540
  Gas price: 8e-09 gwei   Gas limit: 5000000   Nonce: 542
  UniswapV3Pool.burn confirmed (LS)   Block: 675   Gas used: 40630 (0.81%)

Failed to remove liquidty LS
Transaction sent: 0x735c5c6ceb2f98696a21f6e94671269c5b5e49aaf065836a70b4c0e05a664726
  Gas price: 8e-09 gwei   Gas limit: 5000000   Nonce: 543
  UniswapV3Pool.burn confirmed (LS)   Block: 676   Gas used: 40630 (0.81%)

Failed to remove liquidty LS
Episode:1 Score:150
Transaction sent: 0x24c4f4e3a30733cedbf0f782b7d471fb4e0b4e8654ca30fb467eaae19683c9b2
  Gas price: 8e-09 gwei   Gas limit: 5000000   Nonce: 544
  UniswapV3Pool.burn confirmed (LS)   Block: 677   Gas used: 40630 (0.81%)

Failed to remove 

## Train and Evaluate agent Stablebaseline Agents

In [ ]:
from stable_baselines3 import PPO

# Create environment


# Instantiate the agent
model = PPO('MultiInputPolicy', env_box, verbose=1)

# Train the agent
model.learn(total_timesteps=10)

# Evaluate the agent
obs = env_box.reset()
for i in range(5):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    if done:
      obs = env.reset()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AttributeError: 'Box' object has no attribute 'spaces'

## PPO stabebaseline model with Box Spaces

In [ ]:
class DiscreteSimpleEnvBox(Env):
    def __init__(self, price_lower_low, price_lower_high, price_upper_low, price_upper_high, agent_budget_usd, pool):
        super(DiscreteSimpleEnvBox, self).__init__()

        self.pool = pool
        self.global_state = self.pool.get_global_state()

        self.action_space = spaces.Box(
            low=np.array([price_lower_low, price_upper_low], dtype=np.float32),
            high=np.array([price_lower_high, price_upper_high], dtype=np.float32),
            dtype=np.float32
        )

        self.reward = 0
        self.episode_count = 0
        self.step_count = 0
        self.cumulative_reward = 0
        self.done = False

        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 0], dtype=np.float32),
            high=np.array([np.inf, np.inf, np.inf, np.inf], dtype=np.float32),
            dtype=np.float32
        )
        self.agent_budget_usd = agent_budget_usd

    def reset(self):
        self.global_state = self.pool.get_global_state()
        self.state = np.array([
            float(self.global_state['curr_price']),
            float(self.global_state['liquidity_raw']),
            float(self.global_state['feeGrowthGlobal0X128']),
            float(self.global_state['feeGrowthGlobal1X128'])
        ], dtype=np.float32)

        self.done = False
        self.cumulative_reward = 0
        self.episode_count += 1
        self.step_count=0
        print(f"Episode: {self.episode_count}")
        
        return self.state

    def step(self, action):
        _action=self._take_action(action)
        self.state = self.get_obs_space()
        self.reward = self._calculate_reward(action,_action)
        
        self.cumulative_reward += self.reward
        self.done = self._is_done()

        self.step_count += 1
        print(f"Step{self.step_count}_reward: {self.reward} cumulative_reward: {self.cumulative_reward}")

        return self.state, self.reward, self.done, {}

    def _take_action(self, action):
        tick_lower = price_to_valid_tick(action[0])
        tick_upper = price_to_valid_tick(action[1])
        amount = self.agent_budget_usd
        tx_receipt=self.pool.add_liquidity(GOD_ACCOUNT, tick_lower, tick_upper, amount, b'')
        return tx_receipt

    def get_obs_space(self):
        #cmd = "rm -rf outdir_csv; tsp run netlists/uniswapV3/netlist.py outdir_csv"
        #result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        engine.run()

        #print(f'Standard Output:\n{result.stdout}')
        #print(f'Standard Error:\n{result.stderr}')
        
        self.global_state = self.pool.get_global_state()
        self.state = np.array([
            float(self.global_state['curr_price']),
            float(self.global_state['liquidity_raw']),
            float(self.global_state['feeGrowthGlobal0X128']),
            float(self.global_state['feeGrowthGlobal1X128'])
        ], dtype=np.float32)
        return self.state

    def _calculate_reward(self, action,_action):
        tick_lower = price_to_valid_tick(action[0], 60)
        tick_upper = price_to_valid_tick(action[1], 60)
        liquidity = _action.events['Mint']['amount']
        #liquidity = self.pool.budget_to_liquidity(tick_lower, tick_upper, amount)
        self.pool.remove_liquidity_with_liquidty(GOD_ACCOUNT, tick_lower, tick_upper, liquidity)
        _,fee_income = self.pool.collect_fee(GOD_ACCOUNT, tick_lower, tick_upper)
        
        self.reward = fee_income

        return self.reward

    def _is_done(self):
        threshold =1000 
        return self.step_count>4 or self.cumulative_reward>threshold
    
price_lower_low = 1000
price_lower_high = 2000
price_upper_low = 2000
price_upper_high = 3000
agent_budget_usd = 1000
pool = weth_usdc_pool

env_box = DiscreteSimpleEnvBox(price_lower_low, price_lower_high, price_upper_low, price_upper_high, agent_budget_usd,weth_usdc_pool)

In [ ]:
# Instantiate the agent
model = PPO('MlpPolicy', env_box, verbose=0)

/mnt/c/Users/hijaz tr/Desktop/cadCADProject1/tokenspice/tsp_venv/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [ ]:
model.learn(total_timesteps=10)

Episode: 1


{'Mint': [OrderedDict([('sender', '0x330997E70b83f1a562490FCaA5996314fA5a971a'), ('owner', '0x330997E70b83f1a562490FCaA5996314fA5a971a'), ('tickLower', 69060), ('tickUpper', 76020), ('amount', 38029106218509850000), ('amount0', 0), ('amount1', 500000000000255239480)])]}
{'Mint': [OrderedDict([('sender', '0x1eE1Ded283e6fC8A46BBf5387e54f7933Db62d08'), ('owner', '0x1eE1Ded283e6fC8A46BBf5387e54f7933Db62d08'), ('tickLower', 75960), ('tickUpper', 80820), ('amount', 162481706991257550000), ('amount0', 466870961258087394), ('amount1', 695433491919976886337)])]}
amount_swap:54563877973153464
contract token0 ablance:14426575755032498245,contract token1 ablance:47960370962570487081622
{'Transfer': [OrderedDict([('from', '0x4bc0AE6806257DE91D811B718D0671F003B52285'), ('to', '0x4F9a4C220cDB260830917a8CC3b3442117159C15'), ('value', 129808311841901084104)])], 'Swap': [OrderedDict([('sender', '0x4F9a4C220cDB260830917a8CC3b3442117159C15'), ('recipient', '0x4F9a4C220cDB260830917a8CC3b3442117159C15'), ('

KeyboardInterrupt: 

In [ ]:
# Evaluate the agent
obs = env_box.reset()
for i in range(5):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    if done:
      obs = env.reset()


# Presteps

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import requests
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression  # Replace with appropriate ML model
import time
import tensorflow as tf
from gym import spaces
from collections import deque
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\hijaz tr\Desktop\cadCADProject1\abcde\lib\site-packages\google\protobuf\internal\__init__.py)

In [11]:
import requests

UNISWAP_V3_SUBGRAPH_URL = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

def fetch_pool_data(pool_id):
    # Fetching liquidity positions
    liquidity_query = """
    {
      positions(where: { pool: "%s" }) {
        liquidity
        tickLower {
          price0
        }
        tickUpper {
          price0
        }
      }
    }
    """ % pool_id
    liquidity_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': liquidity_query}).json()

    # Fetching pool price, volume, fees, and reserves
    pool_query = """
    {
      pools(where: { id: "%s" }) {
        token0Price
        token1Price
        volumeUSD
        feesUSD
        totalValueLockedToken0
        totalValueLockedToken1
      }
    }
    """ % pool_id
    pool_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': pool_query}).json()

    # Fetching swaps and trades
    swaps_query = """
    {
      swaps(where: { pool: "%s" }, first: 10, orderBy: timestamp, orderDirection: desc) {
        amount0
        amount1
        amountUSD
        timestamp
      }
    }
    """ % pool_id
    swaps_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': swaps_query}).json()

        # Constructing the state representation with float data type
    state = {
        'liquidity_ranges': [(float(position['tickLower']['price0']), float(position['tickUpper']['price0']), float(position['liquidity'])) for position in liquidity_data['data']['positions']],
        'pool_price': float(pool_data['data']['pools'][0]['token0Price']),
        'volume': float(pool_data['data']['pools'][0]['volumeUSD']),
        'fees': float(pool_data['data']['pools'][0]['feesUSD']),
        'reserves': {
            'token0': float(pool_data['data']['pools'][0]['totalValueLockedToken0']),
            'token1': float(pool_data['data']['pools'][0]['totalValueLockedToken1'])
        },
        'recent_swaps': [(float(swap['amount0']), float(swap['amount1']), float(swap['amountUSD'])) for swap in swaps_data['data']['swaps']]
    }

    return state
# Example usage
pool_id = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
state_representation = fetch_pool_data(pool_id)
print(state_representation)



{'liquidity_ranges': [(444741764.1356279, 454634004.0177212, 0.0), (412607821.9483857, 425172963.1539906, 0.0), (384714329.63756645, 434629940.8477182, 0.0), (314037123.69593394, 499939933.7975011, 0.0), (410960769.3130819, 442523715.9566586, 0.0), (415921756.8963616, 439876655.13499963, 0.0), (427304044.68175, 428159465.1361321, 0.0), (426024117.39383185, 426450333.27321035, 0.0), (350901535.2019599, 512595332.7747065, 0.0), (421785317.4774468, 422207292.6489402, 0.0), (419262308.3170634, 419681759.3437395, 0.0), (406871844.7581571, 416337865.8679679, 0.0), (414675923.600571, 415506063.80495846, 746755412055405.0), (363036709.6754226, 487596982.30670816, 0.0), (412607821.9483857, 413020615.4933756, 0.0), (394847556.70360464, 428587817.3244157, 0.0), (333455094.61223024, 499939933.7975011, 0.0), (346023384.3408586, 489551178.31431454, 0.0), (406465196.6034308, 406871844.7581571, 0.0), (407686361.9682095, 408094231.83797157, 0.0), (410550034.4818099, 410960769.3130819, 0.0), (310601815.

# DDPG

In [17]:
class ReplayBuffer:
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape))
        self.new_state_memory = np.zeros((self.mem_size, *input_shape))
        self.action_memory = np.zeros((self.mem_size, n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        dones = self.terminal_memory[batch]

        return states, actions, rewards, states_, dones

class Actor(tf.keras.Model):
    def __init__(self, n_actions):
        super(Actor, self).__init__()
        self.fc1 = tf.keras.layers.Dense(400, activation='relu')
        self.fc2 = tf.keras.layers.Dense(300, activation='relu')
        self.mu = tf.keras.layers.Dense(n_actions, activation='tanh')

    def call(self, state):
        x = self.fc1(state)
        x = self.fc2(x)
        mu = self.mu(x)
        return mu

class Critic(tf.keras.Model):
    def __init__(self, n_actions):
        super(Critic, self).__init__()
        self.fc1 = tf.keras.layers.Dense(400, activation='relu')
        self.fc2 = tf.keras.layers.Dense(300, activation='relu')
        self.q = tf.keras.layers.Dense(1, activation=None)

    def call(self, state, action):
        x = tf.concat([state, action], axis=1)
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.q(x)
        return q
    
class DDPG:
    def __init__(self, alpha=0.001, beta=0.002, input_dims=[8], tau=0.005, env=None,
                 gamma=0.99, n_actions=2, max_size=1000000, layer1_size=400, 
                 layer2_size=300, batch_size=64):
        self.gamma = gamma
        self.tau = tau
        self.memory = ReplayBuffer(max_size, input_dims, n_actions)
        self.batch_size = batch_size

        self.actor = Actor(n_actions=n_actions)
        self.critic = Critic(n_actions=n_actions)

        self.target_actor = Actor(n_actions=n_actions)
        self.target_critic = Critic(n_actions=n_actions)

        self.actor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=alpha))
        self.critic.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=beta))
        self.target_actor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=alpha))
        self.target_critic.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=beta))

        self.update_network_parameters(tau=1)

    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau

        weights = []
        targets = self.target_actor.weights
        for i, weight in enumerate(self.actor.weights):
            weights.append(weight * tau + targets[i] * (1 - tau))
        self.target_actor.set_weights(weights)

        weights = []
        targets = self.target_critic.weights
        for i, weight in enumerate(self.critic.weights):
            weights.append(weight * tau + targets[i] * (1 - tau))
        self.target_critic.set_weights(weights)

    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def choose_action(self, observation):
        state = tf.convert_to_tensor([observation])
        actions = self.actor(state)

        return actions[0].numpy()

    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return

        state, action, reward, new_state, done = \
                                      self.memory.sample_buffer(self.batch_size)

        states = tf.convert_to_tensor(state, dtype=tf.float32)
        states_ = tf.convert_to_tensor(new_state, dtype=tf.float32)
        rewards = tf.convert_to_tensor(reward, dtype=tf.float32)
        actions = tf.convert_to_tensor(action, dtype=tf.float32)

        with tf.GradientTape() as tape:
            target_actions = self.target_actor(states_)
            critic_value_ = tf.squeeze(self.target_critic(
                                states_, target_actions), 1)
            critic_value = tf.squeeze(self.critic(states, actions), 1)
            target = reward + self.gamma*critic_value_*(1-done)
            critic_loss = tf.keras.losses.MSE(target, critic_value)

        critic_network_gradient = tape.gradient(critic_loss, 
                                            self.critic.trainable_variables)
        self.critic.optimizer.apply_gradients(zip(
            critic_network_gradient, self.critic.trainable_variables))

        with tf.GradientTape() as tape:
            new_policy_actions = self.actor(states)
            actor_loss = -self.critic(states, new_policy_actions)
            actor_loss = tf.math.reduce_mean(actor_loss)

        actor_network_gradient = tape.gradient(actor_loss, 
                                    self.actor.trainable_variables)
        self.actor.optimizer.apply_gradients(zip(
            actor_network_gradient, self.actor.trainable_variables))

        self.update_network_parameters()

# Initialize environment and agent
env = UniswapV3Environment(1000,10)
n_actions = env.action_space.shape[0]
input_dims = env.observation_space.shape
agent = DDPG(alpha=0.001, beta=0.002, input_dims=input_dims, tau=0.005, env=env,
             n_actions=n_actions, layer1_size=400, layer2_size=300, batch_size=64)

n_episodes = 1000
max_steps = 500

for i in range(n_episodes):
    state = env.reset()
    episode_reward = 0
    
    for _ in range(max_steps):
        action = agent.choose_action(state)
        # Introduce exploration noise to the action here if needed

        next_state, reward, done, _ = env.step(action)
        
        agent.remember(state, action, reward, next_state, done)
        agent.learn()

        state = next_state
        episode_reward += reward
        
        if done:
            break

    print(f"Episode {i+1}: Reward = {episode_reward}")

# After training, save your agent if needed
# agent.save_model()


# Tabular Q Learning

In [19]:
class LiquidityPool:
    def __init__(self):

        self.state = 50
        # Threshold for maximum and minimum liquidity
        self.max_liquidity = 10000
        self.min_liquidity = 0

    def step(self, action):
        # Increase liquidity
        if action == 0:
            self.state += 10
        # Decrease liquidity
        elif action == 1:
            self.state -= 10
        
        # Ensure that liquidity remains within bounds
        self.state = max(self.min_liquidity, self.state)
        self.state = min(self.max_liquidity, self.state)
        
        # Reward function
        if self.state == self.max_liquidity:
            reward = 1
        elif self.state == self.min_liquidity:
            reward = -1
        else:
            reward = 0

        done = self.state in [self.min_liquidity, self.max_liquidity]
        
        return self.state, reward, done

class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Q = {}

    def choose_action(self, state):
        if np.random.uniform(0, 1) < self.epsilon:
            return np.random.choice([0, 1])
        else:
            return max(list(range(2)), key = lambda x: self.get_Q(state, x))

    def get_Q(self, state, action):
        return self.Q.get((state, action), 0.0)

    def learn(self, state, action, reward, next_state):
        best_next_action = self.choose_action(next_state)
        q_next = self.get_Q(next_state, best_next_action)
        q_curr = self.get_Q(state, action)
        self.Q[(state, action)] = q_curr + self.alpha * (reward + self.gamma * q_next - q_curr)

pool = LiquidityPool()
agent = QLearningAgent()

for episode in range(1000):
    state = pool.state
    total_reward = 0
    done = False

    while not done:
        action = agent.choose_action(state)
        next_state, reward, done = pool.step(action)
        agent.learn(state, action, reward, next_state)
        total_reward += reward
        state = next_state

    if episode % 100 == 0:
        print(f"Episode: {episode}, Total Reward: {total_reward}")

print(agent.Q)

Episode: 0, Total Reward: 1
Episode: 100, Total Reward: 1
Episode: 200, Total Reward: 1
Episode: 300, Total Reward: 1
Episode: 400, Total Reward: 1
Episode: 500, Total Reward: 1
Episode: 600, Total Reward: 1
Episode: 700, Total Reward: 1
Episode: 800, Total Reward: 1
Episode: 900, Total Reward: 1
{(50, 0): 0.0, (60, 0): 0.0, (70, 0): 0.0, (80, 0): 0.0, (90, 0): 0.0, (100, 0): 0.0, (110, 0): 0.0, (120, 0): 0.0, (130, 0): 0.0, (140, 0): 0.0, (150, 0): 0.0, (160, 0): 0.0, (170, 1): 0.0, (170, 0): 0.0, (180, 0): 0.0, (190, 0): 0.0, (200, 0): 0.0, (210, 0): 0.0, (220, 0): 0.0, (230, 0): 0.0, (240, 0): 0.0, (250, 0): 0.0, (260, 0): 0.0, (270, 0): 0.0, (280, 0): 0.0, (290, 0): 0.0, (300, 0): 0.0, (310, 0): 0.0, (320, 0): 0.0, (330, 0): 0.0, (340, 0): 0.0, (350, 0): 0.0, (360, 0): 0.0, (370, 0): 0.0, (380, 0): 0.0, (390, 0): 0.0, (400, 0): 0.0, (410, 0): 0.0, (420, 0): 0.0, (430, 0): 0.0, (440, 0): 0.0, (450, 0): 0.0, (460, 0): 0.0, (470, 0): 0.0, (480, 0): 0.0, (490, 0): 0.0, (500, 0): 0.0, (

# Q learning tabular 2

In [3]:
class LiquidityPoolEnvironment:
    def __init__(self, initial_liquidity, fee_percentage=0.003):
        self.liquidity = initial_liquidity
        self.fee_percentage = fee_percentage
        self.total_fees = 0

    def step(self, action):
        # Simulate trading volume
        volume = np.random.randint(1, 100)
        
        # If action is "add liquidity"
        if action == 0:
            self.liquidity += 10
            fees_earned = volume * self.fee_percentage
        # If action is "remove liquidity"
        else:
            self.liquidity = max(self.liquidity - 10, 10)
            fees_earned = volume * self.fee_percentage * 0.5  # Earn half the fees if liquidity is removed
        
        self.total_fees += fees_earned
        reward = fees_earned  # The reward is the fees earned
        next_state = self.liquidity
        
        return next_state, reward

class QLearningAgent:
    def __init__(self, states, actions, alpha=0.1, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.states = states
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.q_table = np.zeros((states, len(actions)))

    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.actions)
        else:
            return np.argmax(self.q_table[state, :])

    def learn(self, state, action, reward, next_state):
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state, :])
        new_value = (1 - self.alpha) * old_value + self.alpha * (reward + self.gamma * next_max)
        self.q_table[state, action] = new_value

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Simulation
env = LiquidityPoolEnvironment(initial_liquidity=100)
agent = QLearningAgent(states=1000, actions=[0, 1])

for episode in range(100):
    state = env.liquidity
    total_reward = 0
    
    for t in range(100):
        action = agent.choose_action(state)
        next_state, reward = env.step(action)
        agent.learn(state, action, reward, next_state)
        state = next_state
        total_reward += reward
    
    print(f"Episode {episode+1}: Total reward = {total_reward}")

Episode 1: Total reward = 11.536499999999998
Episode 2: Total reward = 11.625000000000007
Episode 3: Total reward = 11.121
Episode 4: Total reward = 10.332000000000003


IndexError: index 1000 is out of bounds for axis 0 with size 1000

# DQN

In [1]:
class LiquidityPoolEnvironment:
    def __init__(self, initial_liquidity, fee_percentage=0.003):
        self.initial_liquidity = initial_liquidity
        self.liquidity = initial_liquidity
        self.fee_percentage = fee_percentage
        self.total_fees = 0
        self.done = False  # Flag to indicate if the episode is done

    def step(self, action):
        # Simulate trading volume
        volume = np.random.randint(1, 100)
        
        # If action is "add liquidity"
        if action == 0:
            self.liquidity += 10
            fees_earned = volume * self.fee_percentage
        # If action is "remove liquidity"
        else:
            self.liquidity = max(self.liquidity - 10, 10)
            fees_earned = volume * self.fee_percentage * 0.5  # Earn half the fees if liquidity is removed
        
        self.total_fees += fees_earned
        reward = fees_earned  # The reward is the fees earned
        next_state = self.liquidity
        
        if self.liquidity >= 200:  # Example termination condition
            self.done = True
        
        return next_state, reward, self.done



class DQNAgent:
    def __init__(self, state_size, action_size, batch_size=32, alpha=0.001, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.state_size = state_size
        self.action_size = action_size
        self.batch_size = batch_size
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=2000)  # Experience replay buffer
        self.model = self._build_model()
        self.target_model = self._build_model()  # Target network

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.alpha))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        else:
            q_values = self.model.predict(state)
            return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = np.array(np.random.sample(self.memory, self.batch_size))
        states = np.vstack(batch[:, 0])
        actions = batch[:, 1].astype(int)
        rewards = batch[:, 2]
        next_states = np.vstack(batch[:, 3])
        dones = batch[:, 4].astype(bool)
        
        target_values = rewards + self.gamma * np.max(self.target_model.predict(next_states), axis=1) * ~dones
        target = self.model.predict(states)
        target[np.arange(self.batch_size), actions] = target_values
        self.model.fit(states, target, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
# Create the environment
env = LiquidityPoolEnvironment(initial_liquidity=100)

# Create the DQN agent
state_size = 1  # The state is just the liquidity value
action_size = 2  # Add liquidity or remove liquidity
dqn_agent = DQNAgent(state_size, action_size)

# Training the DQN agent
EPISODES = 100
for episode in range(EPISODES):
    state = np.array([[env.liquidity]])  # Convert state to 2D array
    total_reward = 0
    
    for t in range(100):
        action = dqn_agent.choose_action(state)
        next_state, reward,done = env.step(action)
        next_state = np.array([[next_state]])
        dqn_agent.remember(state, action, reward, next_state, False)
        state = next_state
        total_reward += reward
        
        # Update DQN agent's Q-values
        dqn_agent.replay()
        
        if env.done:
            break
    
    dqn_agent.update_target_model()
    print(f"Episode {episode+1}: Total reward = {total_reward}")


NameError: name 'deque' is not defined

# Random Models


In [ ]:
class UniswapV3Env1_1(gym.Env):
    def __init__(self, initial_state):
        super(UniswapV3Env1_1, self).__init__()
        
        # State space: [current price, price volatility, TVL, volumeUSD, feesUSD, n_LPs]
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0, 0, 0, 0]), 
                                                high=np.array([np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]), 
                                                dtype=np.float32)
        
        # Action space: [lower tick, upper tick]
        # which_action = gym.spaces.Discrete(3)
        # how_much_action = gym.spaces.Box(low=0,high=np.inf)
        #action_space=gym.spaces.Tuple(which_actio, how_much_action)
        self.action_space = gym.spaces.Box(low=np.array([0, 0]), high=np.array([np.inf, np.inf]), dtype=np.float32)
        
        self.current_state = initial_state
        self.max_days = 30
        self.current_day = 0

    def step(self, action):
        # For simplicity, we can simulate the change in state due to market dynamics as random changes
        # However, using real market data or a more sophisticated model would be better
        self.current_state += np.random.normal(0, 11, size=self.observation_space.shape[0])
        
        # Calculate reward as fees earned within the selected tick range
        # Here we make a simplistic assumption that fees are proportional to volume and the size of the tick range
        reward = self.current_state[3] * 0.03
        
        self.current_day += 1
        done = self.current_day >= self.max_days
        
        return self.current_state, reward, done, {}

    def reset(self):
        self.current_state = np.array([1.0, 0.1, 10000, 1000, 50, 100])  # Example initial values
        self.current_day = 0
        return self.current_state

    def render(self, mode='human'):
        pass


class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(QNetwork, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.model(x)

class DQNAgent:
    def __init__(self, state_dim, action_dim, learning_rate=1e-3, gamma=0.99, epsilon=1.0):
        self.q_network = QNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.gamma = gamma
        self.epsilon = epsilon
        self.action_dim = action_dim
        
    def act(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.rand(2)  # Random action in the range [0, 1] for both ticks
        q_values = self.q_network(torch.FloatTensor(state))
        return q_values.detach().numpy()  # Return Q-values as the action (ticks)

    def learn(self, state, action, reward, next_state):
        state = torch.FloatTensor([state])
        next_state = torch.FloatTensor([next_state])
        reward = torch.FloatTensor([reward])
        
        curr_q = self.q_network(state)
        next_q = self.q_network(next_state).detach().max().item()
        expected_q = reward + self.gamma * next_q

        loss = nn.MSELoss()(curr_q, expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

''''
def train(agent, env, episodes=100):
    total_rewards = []
    
    for episode in range(episodes):
        state = env.reset()
        done = False
        episode_reward = 0
        
        while not done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.learn(state, action, reward, next_state)
            state = next_state
            episode_reward += reward
            #print(f"Lower_tick: {action[0]}, Upper_tick: {action[1]}")
        
        total_rewards.append(episode_reward)
        agent.epsilon = max(agent.epsilon * 0.995, 0.01)  # Decay epsilon
        
        if episode % 5 == 0:
            print(f"Episode: {episode}, Reward: {episode_reward}")

    return total_rewards
'''

def train_with_viz(agent, env, episodes=100):
    fig, axs = plt.subplots(3, 1, figsize=(10, 10))
    
    total_rewards = []
    all_states = []
    all_actions = []
    all_episode_rewards = []

    for episode in range(episodes):
        state = env.reset()
        done = False
        episode_reward = 0
        episode_states = []
        episode_actions = []
        episode_episode_rewards = []
        
        while not done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.learn(state, action, reward, next_state)
            state = next_state
            
            episode_reward += reward
            episode_states.append(state)
            episode_actions.append(action)
            episode_episode_rewards.append(episode_reward)
        
        total_rewards.append(episode_reward)
        agent.epsilon = max(agent.epsilon * 0.995, 0.01)  # Decay epsilon

        all_states.extend(episode_states)
        all_actions.extend(episode_actions)
        all_episode_rewards.extend(episode_episode_rewards)

        if episode % 5 == 0:
            print(f"Episode: {episode}, Reward: {episode_reward}")

    # Plotting the States
    axs[0].plot([s[0] for s in all_states], label='Current Price')
    axs[0].plot([s[1] for s in all_states], label='Price Volatility')
    axs[0].legend()
    axs[0].set_title('States Over Time')

    # Plotting the Actions with shaded area
    axs[1].fill_between(list(range(len(episode_actions))),
                        [a[0] for a in episode_actions],
                        [a[1] for a in episode_actions],
                        color='skyblue', alpha=0.4, label="Liquidity Range")
    axs[1].plot([a[0] for a in episode_actions], label='Lower Tick', color='blue')
    axs[1].plot([a[1] for a in episode_actions], label='Upper Tick', color='orange')
    axs[1].legend()
    axs[1].set_title('Actions Over Time for Last Episode')

    # Plotting the Rewards
    axs[2].plot(all_episode_rewards, label='Episode Reward')
    axs[2].legend()
    axs[2].set_title('Rewards Over Time')

    plt.tight_layout()
    plt.show()

    return total_rewards


def evaluate(agent, env, episodes=100):
    total_rewards = []

    # Set the epsilon of the agent to 0 to disable exploration
    original_epsilon = agent.epsilon
    agent.epsilon = 0

    for _ in range(episodes):
        state = env.reset()
        done = False
        episode_reward = 0

        while not done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            episode_reward += reward

        total_rewards.append(episode_reward)

    # Restore the original epsilon value
    agent.epsilon = original_epsilon

    avg_reward = sum(total_rewards) / episodes
    return avg_reward


In [ ]:
import requests
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
# Define State
def fetch_pool_data(pool_id= "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", UNISWAP_V3_SUBGRAPH_URL = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'):
    # Fetching liquidity positions
    liquidity_query = """
    {
      positions(where: { pool: "%s" }) {
        liquidity
        tickLower {
          price0
        }
        tickUpper {
          price0
        }
      }
    }
    """ % pool_id
    liquidity_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': liquidity_query}).json()

    # Fetching pool price, volume, fees, and reserves
    pool_query = """
    {
      pools(where: { id: "%s" }) {
        token0Price
        token1Price
        volumeUSD
        feesUSD
        totalValueLockedToken0
        totalValueLockedToken1
      }
    }
    """ % pool_id
    pool_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': pool_query}).json()

    # Fetching swaps and trades
    swaps_query = """
    {
      swaps(where: { pool: "%s" }, first: 10, orderBy: timestamp, orderDirection: desc) {
        amount0
        amount1
        amountUSD
        timestamp
      }
    }
    """ % pool_id
    swaps_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': swaps_query}).json()

        # Constructing the state representation with float data type
    state = {
        'liquidity_ranges': [(float(position['tickLower']['price0']), float(position['tickUpper']['price0']), float(position['liquidity'])) for position in liquidity_data['data']['positions']],
        'pool_price': float(pool_data['data']['pools'][0]['token0Price']),
        'volume': float(pool_data['data']['pools'][0]['volumeUSD']),
        'fees': float(pool_data['data']['pools'][0]['feesUSD']),
        'reserves': {
            'token0': float(pool_data['data']['pools'][0]['totalValueLockedToken0']),
            'token1': float(pool_data['data']['pools'][0]['totalValueLockedToken1'])
        },
        'recent_swaps': [(float(swap['amount0']), float(swap['amount1']), float(swap['amountUSD'])) for swap in swaps_data['data']['swaps']]
    }

    return state

state_representation = fetch_pool_data()
print(state_representation)

# Instantiate the environment
current_price=state_representation['pool_price']
price_volatility=0.1
tvl=state_representation['reserves']['token0'] + state_representation['reserves']['token1']*current_price
volumeUSD=state_representation['volume']
feesUSD=state_representation['fees']
n_LPs=tvl/100000
initial_state = np.array([current_price, price_volatility, tvl, volumeUSD, feesUSD, n_LPs])#np.array([1.0, 0.1, 10000, 1000, 50, 100])  # Example initial values
env = UniswapV3Env1_1(initial_state)

# State dimension is 6 (current price, price volatility, TVL, volumeUSD, feesUSD, n_LPs)
# Action dimension is 2 (lower tick, upper tick)
agent = DQNAgent(6, 2)

# Train the agent
rewards = train(agent, env, episodes=100)
# Evaluate the agent
avg_reward = evaluate(agent, env, episodes=10)
print(f"Average reward over 100 episodes: {avg_reward}")
torch.save(agent.q_network.state_dict(), 'trained_model.pth')
model = QNetwork(6, 2)  # Initialize the model architecture
model.load_state_dict(torch.load('trained_model.pth'))
model.eval()  # Set the model to evaluation mode

# Get real time data of pool for state defination, get action in this state from learned agent and apply action in real enviroenmnt 
current_price=state_representation['pool_price']
price_volatility=0.1
tvl=state_representation['reserves']['token0'] + state_representation['reserves']['token1']*current_price
volumeUSD=state_representation['volume']
feesUSD=state_representation['fees']
n_LPs=tvl/100000

state = np.array([current_price, price_volatility, tvl, volumeUSD, feesUSD, n_LPs])
action = agent.act(state)
action


In [ ]:
class UniswapV3Env1_1(gym.Env):
    def __init__(self, initial_state):
        super(UniswapV3Env1_1, self).__init__()
        
        # State space: [current price, price volatility, TVL, volumeUSD, feesUSD, n_LPs]
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0, 0, 0, 0]), 
                                                high=np.array([np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]), 
                                                dtype=np.float32)
        
       
        #action_space=gym.spaces.Tuple(which_actio, how_much_action)
        self.action_space = gym.spaces.Box(low=np.array([0, 0]), high=np.array([1000, 1000]), dtype=np.float32)
        #self.action_space = Dict({'lower_tick':Box(0,1000), "upper_tick":Box(1000,10000)})
        
        self.current_state = initial_state
        self.max_days = 120
        self.current_day = 0

    def step(self, action):
        # For simplicity, we can simulate the change in state due to market dynamics as random changes
        # However, using real market data or a more sophisticated model would be better
        self.current_state += np.random.normal(0, 11, size=self.observation_space.shape[0])
        
        # Calculate reward as fees earned within the selected tick range
        # Here we make a simplistic assumption that fees are proportional to volume and the size of the tick range
        reward = self.current_state[3] * 0.03
        
        self.current_day += 1
        done = self.current_day >= self.max_days
        
        return self.current_state, reward, done, {}

    def reset(self):
        self.current_state = np.array([1.0, 0.1, 10000, 1000, 50, 100])  # Example initial values
        self.current_day = 0
        return self.current_state

    def render(self):
        print('Fuck you')

In [ ]:
env = UniswapV3Env1_1(initial_state)
env.observation_space.sample()
env.action_space.sample()
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=20000)
evaluate_policy(model, env, n_eval_episodes=10, render=True)
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done,info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        print('obs',obs)
        break

In [ ]:
import gym
from gym import spaces
import requests
import numpy as np
import random

import gym
import numpy as np

class UniswapV3Env(gym.Env):
    def __init__(self):
        # State space: [current price, liquidity in current range, total volume, total fees, etc.]
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0, 0]), 
                                                high=np.array([np.inf, np.inf, np.inf, np.inf]), 
                                                dtype=np.float32)
        
        # Action space: [lower tick, upper tick]
        self.action_space = gym.spaces.Box(low=np.array([0, 0]), high=np.array([np.inf, np.inf]), dtype=np.float32)
        
        self.reset()

    def reset(self):
        # Initialize the state with some reasonable values
        self.current_state = np.array([1.0, 1000, 1000, 50])  # Example initial values
        return self.current_state

    def step(self, action):
        # Extract lower and upper ticks from the action
        lower_tick, upper_tick,amount_usd = action

        # Simulate market participants (traders, LPs, arbitrageurs)
       

        # Simulate liquidity provisioning
        self.add_liquidity(lower_tick, upper_tick,amount_usd)
        self.simulate_abm_model(steps=10)

        # Calculate reward based on fees earned within the selected tick range
        reward = self.calculate_reward(action)

        # Check if the episode is done (e.g., based on time or other criteria)
        done = self.check_done()

        return self.current_state, reward, done, {}

    def calculate_reward(self,action):
        # Calculate the reward based on fees earned within the selected tick range
        # Consider factors like volume, liquidity in the current range, fee tier, etc.
        fee=collect_fee(action)
        return fee  # Example calculation

    def check_done(self):
        # Determine if the episode is done (e.g., based on time or other criteria)
        return False  # Example condition

    def render(self, mode='human'):
        pass
import requests

base_url = 'https://api.arbiscan.io/api'
params = {
    'module': 'account',
    'action': 'balance',
    'address': '0x88197EaCf7545B2686715FbBE3DBb0ec725A8514',
    'tag': 'latest',
    'apikey': '1W6Z6MCUXN8GHMEW1AWV7FJRX9YYY6T25K'
}

response = requests.get(base_url, params=params)
data = response.json()

print(data)

base_url = 'https://api.arbiscan.io/api'
params = {
    'module': 'logs',
    'action': 'getLogs',
    'address': '0xC36442b4a4522E871399CD717aBDD847Ab11FE88',
    'fromBlock': '22523653',
    'toBlock': '22524653',
    'page': '1',
    'offset': '1000',
    'apikey': '1W6Z6MCUXN8GHMEW1AWV7FJRX9YYY6T25K'
}

response = requests.get(base_url, params=params)
data = response.json()

print(data)

import requests
# Define State
def fetch_pool_data(pool_id= "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", UNISWAP_V3_SUBGRAPH_URL = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'):
    # Fetching liquidity positions
    liquidity_query = """
    {
      positions(where: { pool: "%s" }) {
        liquidity
        tickLower {
          price0
        }
        tickUpper {
          price0
        }
      }
    }
    """ % pool_id
    liquidity_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': liquidity_query}).json()

    # Fetching pool price, volume, fees, and reserves
    pool_query = """
    {
      pools(where: { id: "%s" }) {
        token0Price
        token1Price
        volumeUSD
        feesUSD
        totalValueLockedToken0
        totalValueLockedToken1
      }
    }
    """ % pool_id
    pool_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': pool_query}).json()

    # Fetching swaps and trades
    swaps_query = """
    {
      swaps(where: { pool: "%s" }, first: 10, orderBy: timestamp, orderDirection: desc) {
        amount0
        amount1
        amountUSD
        timestamp
      }
    }
    """ % pool_id
    swaps_data = requests.post(UNISWAP_V3_SUBGRAPH_URL, json={'query': swaps_query}).json()

        # Constructing the state representation with float data type
    state = {
        'liquidity_ranges': [(float(position['tickLower']['price0']), float(position['tickUpper']['price0']), float(position['liquidity'])) for position in liquidity_data['data']['positions'] if float(position['liquidity']) != 0.0],
        'pool_price': float(pool_data['data']['pools'][0]['token0Price']),
        'volumeUSD': float(pool_data['data']['pools'][0]['volumeUSD']),
        'feesUSD': float(pool_data['data']['pools'][0]['feesUSD']),
        'reserves': {
            'token0': float(pool_data['data']['pools'][0]['totalValueLockedToken0']),
            'token1': float(pool_data['data']['pools'][0]['totalValueLockedToken1'])
        },
        'recent_swaps': [(float(swap['amount0']), float(swap['amount1']), float(swap['amountUSD'])) for swap in swaps_data['data']['swaps']]
    }

    return state

pool_state = fetch_pool_data()
pool_state


class UniswapV3Env(gym.Env):
    def __init__(self):
        # Define action space as an example, you can customize this
        self.action_space = spaces.Discrete(3)
        
        # Define observation space based on your state structure
        # This is just a placeholder; you should tailor it to fit the structure of your state
        self.observation_space = spaces.Dict({
            'liquidity_ranges': spaces.Box(low=0, high=np.inf, shape=(7, 3), dtype=np.float32),
            'pool_price': spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            'volumeUSD': spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            'feesUSD': spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            'reserves': spaces.Dict({
                'token0': spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
                'token1': spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),
            }),
            'recent_swaps': spaces.Box(low=-np.inf, high=np.inf, shape=(10, 3), dtype=np.float32)
        })

    def reset(self):
        # Initialize state by fetching latest pool data
        self.state = fetch_pool_data()
        return self.state

    def step(self, action):
        # Your step logic here
        pass


# Action Spaces

In [6]:
# Constants for minimum and maximum ticks
from gym import spaces
tick_min = -10**6
tick_max = 10**6
lp_positions = {
    "0x9E5360c7624331a785d1f3CDeA8D0EA2ceB42A71": [
        {"tick_lower": 71220, "tick_upper": 76740, "liquidity": 4.795763907841014e+19},
        {"tick_lower": 70920, "tick_upper": 77160, "liquidity": 4.841062996167714e+19},
        {"tick_lower": 71100, "tick_upper": 76980, "liquidity": 5.264262945287719e+19}
    ]
}

# LP budget
lp_budget = 10**6

# Action 0: Do nothing
do_nothing_space =spaces. Discrete(1)

# Action 1: add_liquidity
add_liquidity_space = spaces.Dict({
    'tick_lower': spaces.Box(low=tick_min, high=tick_max, shape=(1,), dtype=int),
    'tick_upper': spaces.Box(low=tick_min, high=tick_max, shape=(1,), dtype=int),
    'amount_usd': spaces.Box(low=0, high=lp_budget, shape=(1,), dtype=float)
})

# Action 2: remove_liquidity (assuming lp_positions is a list of available positions)
# For simplicity, we'll just use a Discrete space to index into lp_positions
remove_liquidity_space =spaces. Discrete(100)  # Assuming up to 100 positions can be maintained

# Combine them into a single action space
action_space = spaces.Tuple((
    spaces.Discrete(3),  # 0 for 'do nothing', 1 for 'add_liquidity', 2 for 'remove_liquidity'
    add_liquidity_space,
    remove_liquidity_space,
))

In [7]:
action_space.sample()

(0,
 OrderedDict([('amount_usd', array([71804.00727666])),
              ('tick_lower', array([-742568])),
              ('tick_upper', array([-597352]))]),
 8)

In [10]:
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete,Space 

min_tick = -88705  # Make sure these are consistent with your specific requirements
max_tick = tick_min
lp_budget = 10000
lp_liq = 5000  # You should define this based on your requirements

action_space = Dict({
    'add_liq': Dict({
        'tick_lower': Box(low=min_tick, high=max_tick, shape=(1,), dtype=int),
        'tick_upper': Box(low=tick_min, high=tick_max, shape=(1,), dtype=int),
        'amount_usd': Box(low=0, high=lp_budget, shape=(1,), dtype=int)
    }),
    'remove_liq': Dict({
        'tick_lower': Box(low=min_tick, high=max_tick, shape=(1,), dtype=int),
        'tick_upper': Box(low=tick_min, high=tick_max, shape=(1,), dtype=int),
        'amount_liq': Box(low=0, high=lp_liq, shape=(1,), dtype=int)
    }),
    'do_nothing': Discrete(1)  # Using 1 here to indicate one possible 'do nothing' action
})


In [11]:
action_space.sample()

OrderedDict([('add_liq',
              OrderedDict([('amount_usd', array([4925])),
                           ('tick_lower', array([-88705])),
                           ('tick_upper', array([-88705]))])),
             ('do_nothing', 0),
             ('remove_liq',
              OrderedDict([('amount_liq', array([3687])),
                           ('tick_lower', array([-88705])),
                           ('tick_upper', array([-88705]))]))])

In [12]:
from gym.spaces import Dict, Box, Discrete, Tuple

min_tick = -88705
max_tick = -min_tick
tick_min = min_tick  # Make sure these are consistent with your specific requirements
tick_max = max_tick
lp_budget = 10000
lp_liq = 5000  # You should define this based on your requirements

# Define the parameters for each action
add_liq_params = Dict({
    'amount_usd': Box(low=0, high=lp_budget, shape=(1,), dtype=int),
    'tick_lower': Box(low=min_tick, high=max_tick, shape=(1,), dtype=int),
    'tick_upper': Box(low=tick_min, high=tick_max, shape=(1,), dtype=int),
})

remove_liq_params = Dict({
    'amount_liq': Box(low=0, high=lp_liq, shape=(1,), dtype=int),
    'tick_lower': Box(low=min_tick, high=max_tick, shape=(1,), dtype=int),
    'tick_upper': Box(low=tick_min, high=tick_max, shape=(1,), dtype=int),
})

do_nothing_params = Discrete(1)  # Using 1 here to indicate one possible 'do nothing' action

# Combine these into a single action space
action_space = Dict({
    'action_type': Discrete(3),  # 0 for 'do_nothing', 1 for 'add_liq', 2 for 'remove_liq'
    'params': Tuple((do_nothing_params, add_liq_params, remove_liq_params))
})


In [38]:
action_space.sample()

OrderedDict([('action_type', 2),
             ('params',
              (0,
               OrderedDict([('amount_usd', array([9970])),
                            ('tick_lower', array([-38149])),
                            ('tick_upper', array([-62975]))]),
               OrderedDict([('amount_liq', array([4645])),
                            ('tick_lower', array([-68053])),
                            ('tick_upper', array([614]))])))])